# Importing libraries and data

In [106]:
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import sklearn
from scipy.stats import zscore

In [107]:
pd.set_option('display.max_columns',False)

In [108]:
df = pd.read_csv('ipl_2008_to_2024.csv')

In [109]:
df.tail()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
260915,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",2,9.5,Kolkata Knight Riders,Sunrisers Hyderabad,SS Iyer,VR Iyer,AK Markram,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260916,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",2,9.6,Kolkata Knight Riders,Sunrisers Hyderabad,VR Iyer,SS Iyer,AK Markram,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260917,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",2,10.1,Kolkata Knight Riders,Sunrisers Hyderabad,VR Iyer,SS Iyer,Shahbaz Ahmed,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260918,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",2,10.2,Kolkata Knight Riders,Sunrisers Hyderabad,SS Iyer,VR Iyer,Shahbaz Ahmed,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260919,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",2,10.3,Kolkata Knight Riders,Sunrisers Hyderabad,VR Iyer,SS Iyer,Shahbaz Ahmed,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning data

In [110]:
df['batting_team'] =  df['batting_team'].replace('Delhi Daredevils','Delhi Capitals')
df['batting_team'] =  df['batting_team'].replace('Rising Pune Supergiant','Rising Pune Supergiants')
df['batting_team'] =  df['batting_team'].replace('Kings XI Punjab','Punjab Kings')
df['batting_team'] =  df['batting_team'].replace('Royal Challengers Bangalore','Royal Challengers Bengaluru')

In [111]:
df['bowling_team'] =  df['bowling_team'].replace('Delhi Daredevils','Delhi Capitals')
df['bowling_team'] =  df['bowling_team'].replace('Rising Pune Supergiant','Rising Pune Supergiants')
df['bowling_team'] =  df['bowling_team'].replace('Kings XI Punjab','Punjab Kings')
df['bowling_team'] =  df['bowling_team'].replace('Royal Challengers Bangalore','Royal Challengers Bengaluru')

In [112]:
#arun jaitley=ferozshahkotla, brabourne, dy patil, vizag, eden gardens,dharamsala, chinnaswamy, chepauk, mohali, 
#uppal, motera = narendra modi = sardar patel, sawai mansingh, wankhede, subrata roy = pune, zayed

In [113]:
df['venue'] = df['venue'].replace('Arun Jaitley Stadium','Arun Jaitley Stadium, Delhi')
df['venue'] = df['venue'].replace('Feroz Shah Kotla','Arun Jaitley Stadium, Delhi')
df['venue'] = df['venue'].replace('Brabourne Stadium','Brabourne Stadium, Mumbai')
df['venue'] = df['venue'].replace('Dr DY Patil Sports Academy','Dr DY Patil Sports Academy, Mumbai')
df['venue'] = df['venue'].replace('Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
                    'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam')
df['venue'] = df['venue'].replace('Eden Gardens','Eden Gardens, Kolkata')
df['venue'] = df['venue'].replace('Himachal Pradesh Cricket Association Stadium',
                    'Himachal Pradesh Cricket Association Stadium, Dharamsala')
df['venue'] = df['venue'].replace('M Chinnaswamy Stadium','M Chinnaswamy Stadium, Bengaluru')
df['venue'] = df['venue'].replace('M.Chinnaswamy Stadium','M Chinnaswamy Stadium, Bengaluru')
df['venue'] = df['venue'].replace('MA Chidambaram Stadium','MA Chidambaram Stadium, Chepauk, Chennai')
df['venue'] = df['venue'].replace('MA Chidambaram Stadium, Chepauk','MA Chidambaram Stadium, Chepauk, Chennai')
df['venue'] = df['venue'].replace('Maharashtra Cricket Association Stadium',
                                  'Maharashtra Cricket Association Stadium, Pune')
df['venue'] = df['venue'].replace('Subrata Roy Sahara Stadium','Maharashtra Cricket Association Stadium, Pune')
df['venue'] = df['venue'].replace('Punjab Cricket Association IS Bindra Stadium',
                                  'Punjab Cricket Association IS Bindra Stadium, Mohali')
df['venue'] = df['venue'].replace('Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh',
                    'Punjab Cricket Association IS Bindra Stadium, Mohali')
df['venue'] = df['venue'].replace('Punjab Cricket Association Stadium, Mohali',
                                  'Punjab Cricket Association IS Bindra Stadium, Mohali')
df['venue'] = df['venue'].replace('Rajiv Gandhi International Stadium',
                                  'Rajiv Gandhi International Stadium, Uppal, Hyderabad')
df['venue'] = df['venue'].replace('Rajiv Gandhi International Stadium, Uppal',
                                  'Rajiv Gandhi International Stadium, Uppal, Hyderabad')
df['venue'] = df['venue'].replace('Sardar Patel Stadium, Motera','Narendra Modi Stadium, Ahmedabad')
df['venue'] = df['venue'].replace('Sawai Mansingh Stadium','Sawai Mansingh Stadium, Jaipur')
df['venue'] = df['venue'].replace('Wankhede Stadium','Wankhede Stadium, Mumbai')
df['venue'] = df['venue'].replace('Sheikh Zayed Stadium','Zayed Cricket Stadium, Abu Dhabi')

In [114]:
df[['venue']].value_counts().sort_index()

venue                                                                
Arun Jaitley Stadium, Delhi                                              21269
Barabati Stadium                                                          1695
Barsapara Cricket Stadium, Guwahati                                        739
Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow     3283
Brabourne Stadium, Mumbai                                                 6526
Buffalo Park                                                               715
De Beers Diamond Oval                                                      726
Dr DY Patil Sports Academy, Mumbai                                        8898
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam        3537
Dubai International Cricket Stadium                                      11229
Eden Gardens, Kolkata                                                    21846
Green Park                                                   

# Season 2024

In [115]:
df_2024 = df[df['season']==2024]

In [116]:
df_2024['current_score'] = df_2024.groupby(['match_id','innings']).agg(current_runs_off_bat = ('runs_off_bat','cumsum')) + df_2024.groupby(['match_id','innings']).agg(current_runs_off_bat = ('extras','cumsum'))
df_2024['player_dismissed_flag'] = df_2024['player_dismissed'].apply(lambda x:0 if pd.isna(x) else 1)
df_2024['current_wickets'] = df_2024.groupby(['match_id','innings']).agg({'player_dismissed_flag':'cumsum'})

In [117]:
df_2024['over_no'] = df_2024['ball'].astype('str').str.split('.').apply(lambda x: x[0])
df_2024['ball_no'] = df_2024['ball'].astype('str').str.split('.').apply(lambda x: x[1])

In [118]:
df_2024.head(20)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,current_score,player_dismissed_flag,current_wickets,over_no,ball_no
243817,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.1,Royal Challengers Bengaluru,Chennai Super Kings,V Kohli,F du Plessis,DL Chahar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1
243818,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.2,Royal Challengers Bengaluru,Chennai Super Kings,V Kohli,F du Plessis,DL Chahar,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,2
243819,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.3,Royal Challengers Bengaluru,Chennai Super Kings,F du Plessis,V Kohli,DL Chahar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,3
243820,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.4,Royal Challengers Bengaluru,Chennai Super Kings,F du Plessis,V Kohli,DL Chahar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,4
243821,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.5,Royal Challengers Bengaluru,Chennai Super Kings,F du Plessis,V Kohli,DL Chahar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,5
243822,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.6,Royal Challengers Bengaluru,Chennai Super Kings,F du Plessis,V Kohli,DL Chahar,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,0,0,0,6
243823,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.7,Royal Challengers Bengaluru,Chennai Super Kings,F du Plessis,V Kohli,DL Chahar,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0,0,7
243824,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,1.1,Royal Challengers Bengaluru,Chennai Super Kings,F du Plessis,V Kohli,TU Deshpande,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,0,0,1,1
243825,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,1.2,Royal Challengers Bengaluru,Chennai Super Kings,F du Plessis,V Kohli,TU Deshpande,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,0,0,1,2
243826,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,1.3,Royal Challengers Bengaluru,Chennai Super Kings,F du Plessis,V Kohli,TU Deshpande,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,0,0,1,3


# Phase 1: Powerplay

# Team Analysis

In [119]:
pp_2024 = df_2024[df_2024['ball']<6]

In [120]:
team_pp_2024_bat =  pp_2024.groupby(['batting_team']).agg({'runs_off_bat':'sum','ball':'count','wides':'count',
                                                               'noballs':'count','extras':'sum',
                                                           'player_dismissed':'count'}) 

In [121]:
team_pp_2024_bat['runs'] = team_pp_2024_bat['runs_off_bat'] + team_pp_2024_bat['extras']
team_pp_2024_bat['ball'] = team_pp_2024_bat['ball'] - team_pp_2024_bat['wides'] -team_pp_2024_bat['noballs']

In [122]:
team_pp_2024_bat.drop(columns={'noballs','wides','extras','runs_off_bat'},inplace=True)

In [123]:
team_pp_2024_bat['avg_powerplay_score'] = team_pp_2024_bat['runs'] / team_pp_2024_bat['ball'] *36
team_pp_2024_bat['true_powerplay_score']=-(sum(team_pp_2024_bat['runs'])/sum(team_pp_2024_bat['ball'])*36)+ team_pp_2024_bat['avg_powerplay_score']

In [124]:
team_pp_2024_bat.sort_values(by='true_powerplay_score',ascending=False)

,ball,player_dismissed,runs,avg_powerplay_score,true_powerplay_score
batting_team,,,,,
Sunrisers Hyderabad,576,29,1073,67.062500,10.266725
Kolkata Knight Riders,504,21,930,66.428571,9.632797
Delhi Capitals,504,28,897,64.071429,7.275654
Royal Challengers Bengaluru,540,21,876,58.400000,1.604225
Mumbai Indians,504,25,789,56.357143,-0.438632
Punjab Kings,504,23,743,53.071429,-3.724346
Chennai Super Kings,504,19,733,52.357143,-4.438632
Rajasthan Royals,540,19,773,51.533333,-5.262441
Lucknow Super Giants,504,26,695,49.642857,-7.152918


In [125]:
team_pp_2024_bat = team_pp_2024_bat.merge((pp_2024[(pp_2024["runs_off_bat"]==6)|
                                                   (pp_2024['runs_off_bat']==4)]).groupby('batting_team').agg({'runs_off_bat':'count'}),on='batting_team')

In [126]:
team_pp_2024_bat.rename(columns={'runs_off_bat':'boundary_count','player_dismissed':'dismissals'},inplace=True)

In [127]:
team_pp_2024_bat['boundary_pct'] = team_pp_2024_bat['boundary_count']/team_pp_2024_bat['ball']*100
team_pp_2024_bat['balls/dismissal'] = team_pp_2024_bat['ball']/team_pp_2024_bat['dismissals']

In [128]:
team_pp_2024_bat['true_boundary_pct'] = -(sum(team_pp_2024_bat['boundary_count'])/sum(team_pp_2024_bat['ball'])*100) + team_pp_2024_bat['boundary_pct']
team_pp_2024_bat['true_balls/dismissal'] = -(sum(team_pp_2024_bat['ball'])/sum(team_pp_2024_bat['dismissals'])) + team_pp_2024_bat['balls/dismissal']

In [129]:
(sum(team_pp_2024_bat['boundary_count'])/sum(team_pp_2024_bat['ball'])*100) 

25.039123630672925

In [130]:
team_pp_2024_bat['pp_score_rank'] = (team_pp_2024_bat['true_powerplay_score'].rank(ascending=False)).astype('int')
team_pp_2024_bat['boundary_pct_rank'] = (team_pp_2024_bat['true_boundary_pct'].rank(ascending=False)).astype('int')
team_pp_2024_bat['dismissal_rate_rank'] = (team_pp_2024_bat['true_balls/dismissal'].rank(ascending=False)).astype('int')

In [131]:
team_pp_2024_bat = team_pp_2024_bat[['runs','ball','dismissals','boundary_count','avg_powerplay_score',
                                     'boundary_pct','balls/dismissal','true_powerplay_score',
                                     'true_boundary_pct','true_balls/dismissal','pp_score_rank','boundary_pct_rank',
                                     'dismissal_rate_rank']]

In [132]:
team_pp_2024_bat

,runs,ball,dismissals,boundary_count,avg_powerplay_score,boundary_pct,balls/dismissal,true_powerplay_score,true_boundary_pct,true_balls/dismissal,pp_score_rank,boundary_pct_rank,dismissal_rate_rank
batting_team,,,,,,,,,,,,,
Chennai Super Kings,733,504,19,109,52.357143,21.626984,26.526316,-4.438632,-3.412140,4.006492,7,8,3
Delhi Capitals,897,504,28,155,64.071429,30.753968,18.000000,7.275654,5.714845,-4.519824,3,1,10
Gujarat Titans,556,432,16,75,46.333333,17.361111,27.000000,-10.462441,-7.678013,4.480176,10,10,2
Kolkata Knight Riders,930,504,21,153,66.428571,30.357143,24.000000,9.632797,5.318019,1.480176,2,2,5
Lucknow Super Giants,695,504,26,103,49.642857,20.436508,19.384615,-7.152918,-4.602616,-3.135208,9,9,9
Mumbai Indians,789,504,25,121,56.357143,24.007937,20.160000,-0.438632,-1.031187,-2.359824,5,6,7
Punjab Kings,743,504,23,123,53.071429,24.404762,21.913043,-3.724346,-0.634362,-0.606780,6,5,6
Rajasthan Royals,773,540,19,128,51.533333,23.703704,28.421053,-5.262441,-1.335420,5.901229,8,7,1
Royal Challengers Bengaluru,876,540,21,141,58.400000,26.111111,25.714286,1.604225,1.071987,3.194462,4,4,4


In [133]:
pp_2024_grouped = pd.DataFrame(pp_2024.groupby(['match_id','batting_team','over_no'])['runs_off_bat'].sum())

In [134]:
pp_2024_grouped.reset_index(inplace=True)

In [135]:
pp_2024_grouped['15+_over'] = pp_2024_grouped['runs_off_bat'].apply(lambda x: 1 if x>=15 else 0)

In [136]:
big_overs = pp_2024_grouped.groupby(["batting_team"]).agg({'15+_over':'sum','over_no':'count'})

In [137]:
big_overs['15+_over_frequency'] = big_overs['15+_over']/big_overs['over_no']*100
big_overs['true_15+_over_frequency'] = -(big_overs['15+_over'].sum()/big_overs['over_no'].sum())*100 + big_overs['15+_over_frequency']
team_pp_2024_bat = team_pp_2024_bat.merge(big_overs[["15+_over_frequency","true_15+_over_frequency"]],on='batting_team',how='left')

In [138]:
team_pp_2024_bat['15+_overs_rank'] = team_pp_2024_bat['15+_over_frequency'].rank(ascending=False).astype('int')
team_pp_2024_bat

,runs,ball,dismissals,boundary_count,avg_powerplay_score,boundary_pct,balls/dismissal,true_powerplay_score,true_boundary_pct,true_balls/dismissal,pp_score_rank,boundary_pct_rank,dismissal_rate_rank,15+_over_frequency,true_15+_over_frequency,15+_overs_rank
batting_team,,,,,,,,,,,,,,,,
Chennai Super Kings,733,504,19,109,52.357143,21.626984,26.526316,-4.438632,-3.412140,4.006492,7,8,3,8.333333,-5.633803,8
Delhi Capitals,897,504,28,155,64.071429,30.753968,18.000000,7.275654,5.714845,-4.519824,3,1,10,21.428571,7.461435,2
Gujarat Titans,556,432,16,75,46.333333,17.361111,27.000000,-10.462441,-7.678013,4.480176,10,10,2,2.777778,-11.189358,9
Kolkata Knight Riders,930,504,21,153,66.428571,30.357143,24.000000,9.632797,5.318019,1.480176,2,2,5,21.428571,7.461435,2
Lucknow Super Giants,695,504,26,103,49.642857,20.436508,19.384615,-7.152918,-4.602616,-3.135208,9,9,9,2.380952,-11.586184,10
Mumbai Indians,789,504,25,121,56.357143,24.007937,20.160000,-0.438632,-1.031187,-2.359824,5,6,7,11.904762,-2.062374,6
Punjab Kings,743,504,23,123,53.071429,24.404762,21.913043,-3.724346,-0.634362,-0.606780,6,5,6,10.714286,-3.252850,7
Rajasthan Royals,773,540,19,128,51.533333,23.703704,28.421053,-5.262441,-1.335420,5.901229,8,7,1,12.222222,-1.744914,5
Royal Challengers Bengaluru,876,540,21,141,58.400000,26.111111,25.714286,1.604225,1.071987,3.194462,4,4,4,16.666667,2.699531,4


In [139]:
px.bar(team_pp_2024_bat,x=team_pp_2024_bat.index,y=['pp_score_rank','boundary_pct_rank',
                                                    'dismissal_rate_rank','15+_overs_rank'])

# Player Analysis

In [140]:
player_pp_2024_bat = pp_2024.groupby(['striker']).agg({'batting_team':'first','runs_off_bat':'sum','ball':'count',
                                                       'wides':'count','player_dismissed':'count'})                                                           

In [141]:
player_pp_2024_bat['dots'] = (pp_2024[(pp_2024["runs_off_bat"]==0) & (pp_2024['wides'].isnull())]).groupby(['striker']).agg({'runs_off_bat':'count'})
player_pp_2024_bat['fours'] = (pp_2024[pp_2024["runs_off_bat"]==4]).groupby(['striker']).agg({'runs_off_bat':'count'})
player_pp_2024_bat['sixes'] = (pp_2024[pp_2024["runs_off_bat"]==6]).groupby(['striker']).agg({'runs_off_bat':'count'})
player_pp_2024_bat['activity'] = (pp_2024[(pp_2024["runs_off_bat"]!=0) & (pp_2024['wides'].isnull())]).groupby(['striker']).agg({'runs_off_bat':'count'})
player_pp_2024_bat['boundary_runs'] = (pp_2024[pp_2024["runs_off_bat"].isin([4,6])]).groupby(['striker']).agg({'runs_off_bat':'sum'})
player_pp_2024_bat['non_boundary_runs'] = (pp_2024[~pp_2024["runs_off_bat"].isin([4,6])]).groupby(['striker']).agg({'runs_off_bat':'sum'})
player_pp_2024_bat['boundary_balls'] = (pp_2024[pp_2024["runs_off_bat"].isin([4,6])]).groupby(['striker']).agg({'ball':'count'})
player_pp_2024_bat['non_boundary_balls'] = (pp_2024[~pp_2024["runs_off_bat"].isin([4,6])]).groupby(['striker']).agg({'ball':'count'})

In [142]:
player_pp_2024_bat.fillna(0,inplace=True)

In [143]:
player_pp_2024_bat['ball'] = player_pp_2024_bat['ball'] - player_pp_2024_bat['wides']

In [144]:
player_pp_2024_bat.drop(columns={'wides'},inplace=True)

In [145]:
player_pp_2024_bat['strike_rate'] = player_pp_2024_bat['runs_off_bat']/player_pp_2024_bat['ball']*100

In [146]:
player_pp_2024_bat['true_strike_rate'] = -sum(player_pp_2024_bat['runs_off_bat'])/sum(player_pp_2024_bat['ball']) *100 + player_pp_2024_bat['strike_rate']

In [147]:
player_pp_2024_bat['dot_pct'] = player_pp_2024_bat['dots']/player_pp_2024_bat['ball']*100

In [148]:
player_pp_2024_bat['boundary_pct']=(player_pp_2024_bat['fours']+player_pp_2024_bat['sixes'])/player_pp_2024_bat['ball']*100

In [149]:
player_pp_2024_bat['activity_rate'] = player_pp_2024_bat['activity']/player_pp_2024_bat['ball']*100

In [150]:
player_pp_2024_bat['boundary_strike_rate'] = player_pp_2024_bat['boundary_runs']/(player_pp_2024_bat['boundary_balls'])*100

In [151]:
player_pp_2024_bat['non_boundary_strike_rate'] = player_pp_2024_bat['non_boundary_runs']/(player_pp_2024_bat['non_boundary_balls'])*100

In [152]:
player_pp_2024_bat['balls_per_dismissal'] = player_pp_2024_bat['ball']/player_pp_2024_bat['player_dismissed']

In [153]:
player_pp_2024_bat['true_dot_pct'] = -player_pp_2024_bat['dot_pct'] + sum(player_pp_2024_bat['dots'])/sum(player_pp_2024_bat['ball'])*100
player_pp_2024_bat['true_boundary_pct'] = player_pp_2024_bat['boundary_pct'] - sum(player_pp_2024_bat['fours']+player_pp_2024_bat['sixes'])/sum(player_pp_2024_bat['ball'])*100
player_pp_2024_bat['true_non_boundary_sr'] = player_pp_2024_bat['non_boundary_strike_rate'] -sum(player_pp_2024_bat['non_boundary_runs'])/sum(player_pp_2024_bat['non_boundary_balls'])*100
player_pp_2024_bat['true_bpd'] = player_pp_2024_bat['balls_per_dismissal'] - sum(player_pp_2024_bat['ball'])/sum(player_pp_2024_bat['player_dismissed'])
player_pp_2024_bat

,batting_team,runs_off_bat,ball,player_dismissed,dots,fours,sixes,activity,boundary_runs,non_boundary_runs,boundary_balls,non_boundary_balls,strike_rate,true_strike_rate,dot_pct,boundary_pct,activity_rate,boundary_strike_rate,non_boundary_strike_rate,balls_per_dismissal,true_dot_pct,true_boundary_pct,true_non_boundary_sr,true_bpd
striker,,,,,,,,,,,,,,,,,,,,,,,,
A Badoni,Lucknow Super Giants,9,16,1,11.0,1.0,0.0,5.0,4.0,5,1.0,15,56.250000,-92.367601,68.750000,6.250000,31.250000,400.000000,33.333333,16.000000,-24.824766,-18.672118,-12.498969,-6.625551
A Manohar,Gujarat Titans,0,6,0,6.0,0.0,0.0,0.0,0.0,0,0.0,6,0.000000,-148.617601,100.000000,0.000000,0.000000,NaN,0.000000,inf,-56.074766,-24.922118,-45.832303,inf
A Raghuvanshi,Kolkata Knight Riders,78,49,3,21.0,9.0,4.0,28.0,60.0,18,13.0,37,159.183673,10.566072,42.857143,26.530612,57.142857,461.538462,48.648649,16.333333,1.068091,1.608494,2.816346,-6.292217
AA Kulkarni,Lucknow Super Giants,9,8,2,5.0,2.0,0.0,3.0,8.0,1,2.0,6,112.500000,-36.117601,62.500000,25.000000,37.500000,400.000000,16.666667,4.000000,-18.574766,0.077882,-29.165636,-18.625551
AK Markram,Sunrisers Hyderabad,57,45,3,19.0,10.0,0.0,26.0,40.0,17,10.0,36,126.666667,-21.950935,42.222222,22.222222,57.777778,400.000000,47.222222,15.000000,1.703011,-2.699896,1.389919,-7.625551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V Shankar,Gujarat Titans,8,8,0,5.0,0.0,1.0,3.0,6.0,2,1.0,7,100.000000,-48.617601,62.500000,12.500000,37.500000,600.000000,28.571429,inf,-18.574766,-12.422118,-17.260874,inf
VR Iyer,Kolkata Knight Riders,111,50,1,17.0,17.0,5.0,33.0,98.0,13,22.0,29,222.000000,73.382399,34.000000,44.000000,66.000000,445.454545,44.827586,50.000000,9.925234,19.077882,-1.004717,27.374449
WG Jacks,Royal Challengers Bengaluru,73,49,2,22.0,6.0,5.0,27.0,54.0,19,11.0,38,148.979592,0.361991,44.897959,22.448980,55.102041,490.909091,50.000000,24.500000,-0.972726,-2.473139,4.167697,1.874449


# Useful Metrics

In [154]:
player_pp_2024_bat_final = player_pp_2024_bat[['batting_team','runs_off_bat','ball','player_dismissed','strike_rate',
                                               'true_strike_rate','dot_pct','true_dot_pct','boundary_pct',
                                               'true_boundary_pct','non_boundary_strike_rate','true_non_boundary_sr',
                                               'balls_per_dismissal','true_bpd']] 

In [155]:
player_pp_2024_bat_final

,batting_team,runs_off_bat,ball,player_dismissed,strike_rate,true_strike_rate,dot_pct,true_dot_pct,boundary_pct,true_boundary_pct,non_boundary_strike_rate,true_non_boundary_sr,balls_per_dismissal,true_bpd
striker,,,,,,,,,,,,,,
A Badoni,Lucknow Super Giants,9,16,1,56.250000,-92.367601,68.750000,-24.824766,6.250000,-18.672118,33.333333,-12.498969,16.000000,-6.625551
A Manohar,Gujarat Titans,0,6,0,0.000000,-148.617601,100.000000,-56.074766,0.000000,-24.922118,0.000000,-45.832303,inf,inf
A Raghuvanshi,Kolkata Knight Riders,78,49,3,159.183673,10.566072,42.857143,1.068091,26.530612,1.608494,48.648649,2.816346,16.333333,-6.292217
AA Kulkarni,Lucknow Super Giants,9,8,2,112.500000,-36.117601,62.500000,-18.574766,25.000000,0.077882,16.666667,-29.165636,4.000000,-18.625551
AK Markram,Sunrisers Hyderabad,57,45,3,126.666667,-21.950935,42.222222,1.703011,22.222222,-2.699896,47.222222,1.389919,15.000000,-7.625551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V Shankar,Gujarat Titans,8,8,0,100.000000,-48.617601,62.500000,-18.574766,12.500000,-12.422118,28.571429,-17.260874,inf,inf
VR Iyer,Kolkata Knight Riders,111,50,1,222.000000,73.382399,34.000000,9.925234,44.000000,19.077882,44.827586,-1.004717,50.000000,27.374449
WG Jacks,Royal Challengers Bengaluru,73,49,2,148.979592,0.361991,44.897959,-0.972726,22.448980,-2.473139,50.000000,4.167697,24.500000,1.874449


In [156]:
player_pp_2024_bat_final[['runs_off_bat','ball']].describe()

,runs_off_bat,ball
count,83.000000,83.000000
mean,91.963855,61.879518
std,103.484822,62.043336
min,0.000000,2.000000
25%,12.500000,11.500000
50%,44.000000,40.000000
75%,140.000000,102.000000
max,402.000000,231.000000


In [157]:
player_pp_2024_bat_final = (player_pp_2024_bat_final[player_pp_2024_bat_final['ball']>=40])
player_pp_2024_bat_final.head()

,batting_team,runs_off_bat,ball,player_dismissed,strike_rate,true_strike_rate,dot_pct,true_dot_pct,boundary_pct,true_boundary_pct,non_boundary_strike_rate,true_non_boundary_sr,balls_per_dismissal,true_bpd
striker,,,,,,,,,,,,,,
A Raghuvanshi,Kolkata Knight Riders,78,49,3,159.183673,10.566072,42.857143,1.068091,26.530612,1.608494,48.648649,2.816346,16.333333,-6.292217
AK Markram,Sunrisers Hyderabad,57,45,3,126.666667,-21.950935,42.222222,1.703011,22.222222,-2.699896,47.222222,1.389919,15.000000,-7.625551
AM Rahane,Chennai Super Kings,145,114,6,127.192982,-21.424619,46.491228,-2.565994,19.298246,-5.623873,50.000000,4.167697,19.000000,-3.625551
Abhishek Sharma,Sunrisers Hyderabad,343,169,9,202.958580,54.340979,34.911243,9.013991,33.136095,8.213976,49.593496,3.761193,18.777778,-3.847773
Abishek Porel,Delhi Capitals,140,89,4,157.303371,8.685770,44.943820,-1.018587,26.966292,2.044174,42.253521,-3.578782,22.250000,-0.375551


# Metrics used (all in relative form): Strike Rate, Boundary,Non Boundary,Dot PCT and Balls per Dismissal

In [158]:
player_pp_2024_bat_final[["batting_team","runs_off_bat","ball","player_dismissed","strike_rate",
                        "true_strike_rate"]].sort_values(by='strike_rate',ascending=False).reset_index().head(20)

,striker,batting_team,runs_off_bat,ball,player_dismissed,strike_rate,true_strike_rate
0,J Fraser-McGurk,Delhi Capitals,266,106,5,250.943396,102.325795
1,VR Iyer,Kolkata Knight Riders,111,50,1,222.000000,73.382399
2,TM Head,Sunrisers Hyderabad,402,193,6,208.290155,59.672554
3,Abhishek Sharma,Sunrisers Hyderabad,343,169,9,202.958580,54.340979
4,PD Salt,Kolkata Knight Riders,296,160,8,185.000000,36.382399
5,RR Rossouw,Punjab Kings,82,46,3,178.260870,29.643268
6,SP Narine,Kolkata Knight Riders,281,165,5,170.303030,21.685429
7,Ishan Kishan,Mumbai Indians,239,148,8,161.486486,12.868885
8,V Kohli,Royal Challengers Bengaluru,373,231,4,161.471861,12.854260
9,P Simran Singh,Punjab Kings,236,148,7,159.459459,10.841858


In [159]:
player_pp_2024_bat_final[["batting_team","runs_off_bat","ball","player_dismissed","boundary_pct",
                        "true_boundary_pct"]].sort_values(by='boundary_pct',ascending=False).reset_index().head(20)

,striker,batting_team,runs_off_bat,ball,player_dismissed,boundary_pct,true_boundary_pct
0,J Fraser-McGurk,Delhi Capitals,266,106,5,49.056604,24.134485
1,VR Iyer,Kolkata Knight Riders,111,50,1,44.000000,19.077882
2,RR Rossouw,Punjab Kings,82,46,3,39.130435,14.208316
3,TM Head,Sunrisers Hyderabad,402,193,6,38.341969,13.419851
4,Abhishek Sharma,Sunrisers Hyderabad,343,169,9,33.136095,8.213976
5,PD Salt,Kolkata Knight Riders,296,160,8,33.125000,8.202882
6,PP Shaw,Delhi Capitals,140,88,5,30.681818,5.759700
7,SP Narine,Kolkata Knight Riders,281,165,5,30.303030,5.380912
8,F du Plessis,Royal Challengers Bengaluru,309,194,11,28.865979,3.943861
9,YBK Jaiswal,Rajasthan Royals,282,183,8,28.415301,3.493182


In [160]:
player_pp_2024_bat_final[["batting_team","runs_off_bat","ball","player_dismissed","dot_pct",
                        "true_dot_pct"]].sort_values(by='dot_pct',ascending=True).reset_index().head(20)

,striker,batting_team,runs_off_bat,ball,player_dismissed,dot_pct,true_dot_pct
0,TM Head,Sunrisers Hyderabad,402,193,6,33.160622,10.764612
1,VR Iyer,Kolkata Knight Riders,111,50,1,34.000000,9.925234
2,DJ Mitchell,Chennai Super Kings,97,73,2,34.246575,9.678658
3,Abhishek Sharma,Sunrisers Hyderabad,343,169,9,34.911243,9.013991
4,V Kohli,Royal Challengers Bengaluru,373,231,4,36.363636,7.561597
5,J Fraser-McGurk,Delhi Capitals,266,106,5,36.792453,7.132781
6,RD Gaikwad,Chennai Super Kings,261,185,5,37.837838,6.087396
7,JM Bairstow,Punjab Kings,169,117,6,39.316239,4.608994
8,JC Buttler,Rajasthan Royals,198,149,5,39.597315,4.327918
9,PP Shaw,Delhi Capitals,140,88,5,39.772727,4.152506


In [161]:
player_pp_2024_bat_final[["batting_team","runs_off_bat","ball","player_dismissed","non_boundary_strike_rate",
                "true_non_boundary_sr"]].sort_values(by='non_boundary_strike_rate',ascending=False).reset_index().head(20)

,striker,batting_team,runs_off_bat,ball,player_dismissed,non_boundary_strike_rate,true_non_boundary_sr
0,DJ Mitchell,Chennai Super Kings,97,73,2,66.129032,20.296730
1,B Sai Sudharsan,Gujarat Titans,164,142,2,58.461538,12.629236
2,JC Buttler,Rajasthan Royals,198,149,5,58.267717,12.435414
3,RD Gaikwad,Chennai Super Kings,261,185,5,57.823129,11.990827
4,V Kohli,Royal Challengers Bengaluru,373,231,4,56.593407,10.761104
5,MA Agarwal,Sunrisers Hyderabad,59,52,3,54.347826,8.515523
6,Shubman Gill,Gujarat Titans,194,148,5,54.098361,8.266058
7,KL Rahul,Lucknow Super Giants,273,207,4,53.370787,7.538484
8,Nithish Kumar Reddy,Sunrisers Hyderabad,37,40,1,52.500000,6.667697
9,SM Curran,Punjab Kings,41,43,1,52.500000,6.667697


In [162]:
player_pp_2024_bat_final[["batting_team","runs_off_bat","ball","player_dismissed","balls_per_dismissal",
                        "true_bpd"]].sort_values(by='balls_per_dismissal',ascending=False).reset_index().head(20)

,striker,batting_team,runs_off_bat,ball,player_dismissed,balls_per_dismissal,true_bpd
0,R Parag,Rajasthan Royals,50,40,0,inf,inf
1,B Sai Sudharsan,Gujarat Titans,164,142,2,71.000000,48.374449
2,V Kohli,Royal Challengers Bengaluru,373,231,4,57.750000,35.124449
3,KL Rahul,Lucknow Super Giants,273,207,4,51.750000,29.124449
4,VR Iyer,Kolkata Knight Riders,111,50,1,50.000000,27.374449
5,Tilak Varma,Mumbai Indians,40,44,1,44.000000,21.374449
6,SM Curran,Punjab Kings,41,43,1,43.000000,20.374449
7,Nithish Kumar Reddy,Sunrisers Hyderabad,37,40,1,40.000000,17.374449
8,RD Gaikwad,Chennai Super Kings,261,185,5,37.000000,14.374449
9,DJ Mitchell,Chennai Super Kings,97,73,2,36.500000,13.874449


In [163]:
# kkr batters have dominated all metrics except non boundary sr (single/double taking capability: which means pp emphasis
# is not on taking those, it is on scoring most number of boundaries as we saw in the team analysis as well )

# Powerplay Case Study: Narine, Salt, V Iyer

In [164]:
#take +ve points of Narine/Salt/Iyer
#targeting specific ball of the over: Narine 6th ball, salt xth ball, pship impact: size, sr, 4s, 6s; 
#salt/narine pship vs all teams op pship

In [165]:
kkr_pp= pp_2024[pp_2024["batting_team"]=='Kolkata Knight Riders']

In [166]:
kkr_pp_narine = kkr_pp[(kkr_pp["striker"]=='SP Narine') & (kkr_pp["wides"].isna())]
narine_pp_bn = kkr_pp_narine.groupby(['ball_no']).agg({'runs_off_bat':'sum','ball':'count','player_dismissed':'count'})
narine_pp_bn['strike_rate'] = narine_pp_bn['runs_off_bat']/narine_pp_bn['ball']*100
narine_pp_bn['bpd'] = narine_pp_bn['ball']/narine_pp_bn['player_dismissed']

In [167]:
kkr_pp_bn=kkr_pp[(kkr_pp["striker"]!='SP Narine')&(kkr_pp["wides"].isna())].groupby(["ball_no"]).agg({'runs_off_bat':'sum',
'ball':'count'})
kkr_pp_bn['strike_rate'] = kkr_pp_bn['runs_off_bat']/kkr_pp_bn['ball']*100

In [168]:
pp_2024_bn=pp_2024[(pp_2024["wides"].isna())&(pp_2024['striker']!='SP Narine')].groupby(['ball_no']).agg({'runs_off_bat':'sum',
                                                                                                        'ball':'count'})
pp_2024_bn['strike_rate'] = pp_2024_bn['runs_off_bat']/pp_2024_bn['ball']*100

In [169]:
narine_pp_bn =narine_pp_bn.merge(kkr_pp_bn,right_index=True,left_index=True,how='left',suffixes=('_narine','_kkr'))

In [170]:
narine_pp_bn = narine_pp_bn.merge(pp_2024_bn,right_index=True,left_index=True,how='left')
narine_pp_bn = narine_pp_bn[narine_pp_bn["ball_narine"]>20]
narine_pp_bn

,runs_off_bat_narine,ball_narine,player_dismissed,strike_rate_narine,bpd,runs_off_bat_kkr,ball_kkr,strike_rate_kkr,runs_off_bat,ball,strike_rate
ball_no,,,,,,,,,,,
1,33,26,1,126.923077,26.000000,94,57,164.912281,1149,782,146.930946
2,42,26,3,161.538462,8.666667,92,56,164.285714,1267,800,158.375000
3,40,23,1,173.913043,23.000000,106,58,182.758621,1151,805,142.981366
4,41,25,0,164.000000,inf,105,54,194.444444,1166,797,146.298620
5,47,28,0,167.857143,inf,87,51,170.588235,1151,799,144.055069
6,59,27,0,218.518519,inf,76,52,146.153846,1199,795,150.817610


In [171]:
salt_pp_bn =kkr_pp[(kkr_pp["striker"]=='PD Salt')&(kkr_pp["wides"].isna())].groupby(["ball_no"]).agg({'runs_off_bat':'sum',
'ball':'count','player_dismissed':'count'})
salt_pp_bn['strike_rate'] = salt_pp_bn['runs_off_bat']/salt_pp_bn['ball']*100

In [172]:
kkr_pp_bn_2=kkr_pp[(kkr_pp["striker"]!='PD Salt')&(kkr_pp["wides"].isna())].groupby(["ball_no"]).agg({'runs_off_bat':'sum',
'ball':'count'})
kkr_pp_bn_2['strike_rate'] = kkr_pp_bn_2['runs_off_bat']/kkr_pp_bn_2['ball']*100

In [173]:
pp_2024_bn_2=pp_2024[(pp_2024["wides"].isna())&(pp_2024['striker']!='PD Salt')].groupby(['ball_no']).agg({'runs_off_bat':'sum',
                                                                                                        'ball':'count'})
pp_2024_bn_2['strike_rate'] = pp_2024_bn_2['runs_off_bat']/pp_2024_bn_2['ball']*100

In [174]:
salt_pp_bn =salt_pp_bn.merge(kkr_pp_bn_2,right_index=True,left_index=True,how='left',suffixes=('_salt','_kkr'))

In [175]:
salt_pp_bn = salt_pp_bn.merge(pp_2024_bn_2,right_index=True,left_index=True,how='left')

In [176]:
salt_pp_bn = salt_pp_bn[salt_pp_bn["ball_salt"]>5]
salt_pp_bn

,runs_off_bat_salt,ball_salt,player_dismissed,strike_rate_salt,runs_off_bat_kkr,ball_kkr,strike_rate_kkr,runs_off_bat,ball,strike_rate
ball_no,,,,,,,,,,
1,58,35,1,165.714286,69.0,48.0,143.750000,1124,773,145.407503
2,49,32,2,153.125000,85.0,50.0,170.000000,1260,794,158.690176
3,48,27,2,177.777778,98.0,54.0,181.481481,1143,801,142.696629
4,47,23,0,204.347826,99.0,56.0,176.785714,1160,799,145.181477
5,57,20,2,285.000000,77.0,59.0,130.508475,1141,807,141.387856
6,37,18,0,205.555556,98.0,61.0,160.655738,1221,804,151.865672


In [177]:
salt_narine=df_2024[(df_2024["striker"].isin(["PD Salt","SP Narine"]))&(df_2024["non_striker"].isin(["PD Salt","SP Narine"]))]
salt_narine_mbm = salt_narine.groupby(["match_id"]).agg({'runs_off_bat':'sum','ball':'count','wides':'count'})
salt_narine_mbm['ball'] = salt_narine_mbm['ball'] - salt_narine_mbm['wides']
salt_narine_mbm

,runs_off_bat,ball,wides
match_id,,,
1422121,21,12,2
1422128,76,39,1
1422134,52,27,2
1426260,0,1,0
1426266,12,11,2
1426269,17,21,0
1426274,53,26,0
1426280,131,63,6
1426285,75,38,1


In [178]:
salt_narine_mbm.drop(columns={'wides'},inplace=True)

In [179]:
salt_narine_mbm["run_rate"] = salt_narine_mbm["runs_off_bat"]/salt_narine_mbm["ball"]*6
total_runs = salt_narine_mbm["runs_off_bat"].sum()
balls = salt_narine_mbm["ball"].sum()
dismissals = len(salt_narine_mbm)
run_rate = total_runs/balls*6
avg_pship = salt_narine_mbm["runs_off_bat"].mean()
median_pship = salt_narine_mbm["runs_off_bat"].median()
salt_narine_mbm['match_number'] = range(1,len(salt_narine_mbm)+1)
sixes = len(salt_narine[salt_narine["runs_off_bat"]==6])
fours = len(salt_narine[salt_narine["runs_off_bat"]==4])
bdry_fq = balls/(sixes+fours)
avg_pship,median_pship, sixes, fours

(42.333333333333336, 36.5, 32, 58)

In [180]:
not_salt_narine = df_2024[(~df_2024['striker'].isin(["PD Salt","SP Narine"])) & (~df_2024['non_striker'].isin(["PD Salt","SP Narine"]))]
op_pship = not_salt_narine[(not_salt_narine["bowling_team"]=='Kolkata Knight Riders') & (not_salt_narine["current_wickets"]==0)].groupby(["match_id"]).agg({'runs_off_bat':'sum','ball':'count','wides':'count'})
op_pship["ball"] = op_pship["ball"]  - op_pship["wides"]
op_pship.drop(columns={'wides'},inplace=True)
op_pship["run_rate"] = op_pship["runs_off_bat"]/op_pship["ball"]*6
op_pship["runs_off_bat"].mean()

27.0

In [181]:
op_condition = (df_2024[(df_2024["current_wickets"]==0) | ((df_2024["current_wickets"]==1)&(df_2024["player_dismissed_flag"]==1))])
op_pship_teams = op_condition.groupby(["batting_team"]).agg({'runs_off_bat':'sum','ball':'count',
                                                             'wides':'count','player_dismissed':'count'})
op_pship_teams['ball'] = op_pship_teams['ball']-op_pship_teams['wides']
op_pship_teams.drop(columns={'wides'},inplace=True)
op_pship_teams['sixes']=op_condition[op_condition["runs_off_bat"]==6].groupby(["batting_team"]).agg({'runs_off_bat':'count'})
op_pship_teams['fours']=op_condition[op_condition["runs_off_bat"]==4].groupby(["batting_team"]).agg({'runs_off_bat':'count'})
op_pship_teams['run_rate']=op_pship_teams['runs_off_bat']/op_pship_teams['ball']*6
op_pship_teams['bdry_freq'] = op_pship_teams['ball']/(op_pship_teams['sixes']+op_pship_teams['fours'])
op_pship_teams['avg_pship'] = op_pship_teams['runs_off_bat']/op_pship_teams['player_dismissed']
op_pship_teams['median_pship'] = (op_condition.groupby(["batting_team","match_id"]).agg({'current_score':'max'}).reset_index()).groupby(["batting_team"]).agg({'current_score':'median'})
op_pship_teams

#in power bi,add another row for salt&narine

,runs_off_bat,ball,player_dismissed,sixes,fours,run_rate,bdry_freq,avg_pship,median_pship
batting_team,,,,,,,,,
Chennai Super Kings,282,231,14,6,37,7.324675,5.372093,20.142857,15.5
Delhi Capitals,478,277,14,24,63,10.353791,3.183908,34.142857,24.5
Gujarat Titans,487,334,12,21,47,8.748503,4.911765,40.583333,28.5
Kolkata Knight Riders,553,301,14,35,63,11.023256,3.071429,39.500000,33.5
Lucknow Super Giants,318,227,14,14,34,8.405286,4.729167,22.714286,10.5
Mumbai Indians,522,328,14,29,60,9.548780,3.685393,37.285714,30.5
Punjab Kings,463,307,14,21,55,9.048860,4.039474,33.071429,18.0
Rajasthan Royals,387,304,15,6,57,7.638158,4.825397,25.800000,22.0
Royal Challengers Bengaluru,680,426,15,34,72,9.577465,4.018868,45.333333,40.0


In [182]:
op_pship_teams.loc['Salt & Narine'] = [total_runs,balls,dismissals,sixes,fours,run_rate,bdry_fq,avg_pship,median_pship]

In [183]:
op_pship_teams

,runs_off_bat,ball,player_dismissed,sixes,fours,run_rate,bdry_freq,avg_pship,median_pship
batting_team,,,,,,,,,
Chennai Super Kings,282.0,231.0,14.0,6.0,37.0,7.324675,5.372093,20.142857,15.5
Delhi Capitals,478.0,277.0,14.0,24.0,63.0,10.353791,3.183908,34.142857,24.5
Gujarat Titans,487.0,334.0,12.0,21.0,47.0,8.748503,4.911765,40.583333,28.5
Kolkata Knight Riders,553.0,301.0,14.0,35.0,63.0,11.023256,3.071429,39.500000,33.5
Lucknow Super Giants,318.0,227.0,14.0,14.0,34.0,8.405286,4.729167,22.714286,10.5
Mumbai Indians,522.0,328.0,14.0,29.0,60.0,9.548780,3.685393,37.285714,30.5
Punjab Kings,463.0,307.0,14.0,21.0,55.0,9.048860,4.039474,33.071429,18.0
Rajasthan Royals,387.0,304.0,15.0,6.0,57.0,7.638158,4.825397,25.800000,22.0
Royal Challengers Bengaluru,680.0,426.0,15.0,34.0,72.0,9.577465,4.018868,45.333333,40.0


In [184]:
#for csv: salt_pp_bn narine_pp_bn, op_pshp_teams

# Phase 2: Middle Overs

# Team Analysis

In [185]:
df_2024_mid = df_2024[(df_2024["ball"]>=6) & (df_2024["ball"]<=15)]

In [186]:
team_mid_2024_bat = df_2024_mid.groupby(["batting_team"]).agg({'runs_off_bat':'sum','ball':'count','wides':'count',
                                                               'noballs':'count','extras':'sum',
                                                           'player_dismissed':'count'})
team_mid_2024_bat['runs'] = team_mid_2024_bat['runs_off_bat'] + team_mid_2024_bat['extras']
team_mid_2024_bat['balls'] = team_mid_2024_bat['ball'] - team_mid_2024_bat['wides'] - team_mid_2024_bat['noballs']

In [187]:
team_mid_2024_bat['avg_score'] = team_mid_2024_bat['runs']/team_mid_2024_bat['balls']*54
team_mid_2024_bat['bpd'] = team_mid_2024_bat['balls']/team_mid_2024_bat['player_dismissed']

In [188]:
team_mid_2024_bat['dots'] = (df_2024_mid[(df_2024_mid["runs_off_bat"]==0) & 
             (df_2024_mid["extras"]==0)]).groupby(['batting_team']).agg({'runs_off_bat':'count'})

team_mid_2024_bat['sixes'] = df_2024_mid[(df_2024_mid["runs_off_bat"]==6)].groupby(['batting_team']).agg({'runs_off_bat':
                                                                                                          'count'})
team_mid_2024_bat['fours'] = df_2024_mid[(df_2024_mid["runs_off_bat"]==4)].groupby(['batting_team']).agg({'runs_off_bat':
                                                                                                          'count'})
team_mid_2024_bat['activity'] = df_2024_mid[(df_2024_mid["runs_off_bat"]!=0) & 
                                (df_2024_mid['extras']==0)].groupby(['batting_team']).agg({'runs_off_bat':'count'})

In [189]:
team_mid_2024_bat['six_frequency'] = team_mid_2024_bat['balls']/team_mid_2024_bat['sixes']
team_mid_2024_bat['four_frequency'] = team_mid_2024_bat['balls']/team_mid_2024_bat['fours']

In [190]:
team_mid_2024_bat['activity_pct'] = team_mid_2024_bat['activity']/team_mid_2024_bat['balls']*100

In [191]:
team_mid_2024_bat.drop(columns={'wides','noballs','runs_off_bat','ball','extras'},inplace=True)

In [192]:
team_mid_2024_bat = team_mid_2024_bat[["runs","balls","player_dismissed","dots","sixes","fours","activity",
                                       "avg_score","bpd","activity_pct",
                   "four_frequency","six_frequency"]]

In [193]:
team_mid_2024_bat['z_score_activity_pct']=(team_mid_2024_bat['activity_pct'] - team_mid_2024_bat['activity_pct'].mean())/team_mid_2024_bat['activity_pct'].std()
team_mid_2024_bat['z_score_four_frequency']=-(team_mid_2024_bat['four_frequency'] - team_mid_2024_bat['four_frequency'].mean())/team_mid_2024_bat['four_frequency'].std() 
team_mid_2024_bat['z_score_six_frequency']=-(team_mid_2024_bat['six_frequency'] - team_mid_2024_bat['six_frequency'].mean())/team_mid_2024_bat['six_frequency'].std() 
team_mid_2024_bat['z_score_avg_score']=(team_mid_2024_bat['avg_score'] - team_mid_2024_bat['avg_score'].mean())/team_mid_2024_bat['activity_pct'].std() 
team_mid_2024_bat['z_score_bpd']=(team_mid_2024_bat['bpd'] - team_mid_2024_bat['bpd'].mean())/team_mid_2024_bat['bpd'].std() 

In [194]:
team_mid_2024_bat['z_performance_index'] = (0.15*team_mid_2024_bat['z_score_activity_pct']) + (0.25*team_mid_2024_bat['z_score_four_frequency']) + (0.25*team_mid_2024_bat['z_score_six_frequency']) + (0.25*team_mid_2024_bat['z_score_avg_score']) + (0.1*team_mid_2024_bat['z_score_bpd']) *100
team_mid_2024_bat.sort_values(by='z_performance_index',ascending=False)

,runs,balls,player_dismissed,dots,sixes,fours,activity,avg_score,bpd,activity_pct,four_frequency,six_frequency,z_score_activity_pct,z_score_four_frequency,z_score_six_frequency,z_score_avg_score,z_score_bpd,z_performance_index
batting_team,,,,,,,,,,,,,,,,,,
Kolkata Knight Riders,1175,721,28,213,58,91,499,88.002774,25.750000,69.209431,7.923077,12.431034,-0.160927,1.479493,0.769722,3.573746,1.950420,20.935805
Rajasthan Royals,1155,810,34,231,51,74,572,77.000000,23.823529,70.617284,10.945946,15.882353,0.426391,-0.400518,-0.204885,-1.016313,1.114493,10.803462
Lucknow Super Giants,1120,756,34,193,47,73,558,80.000000,22.235294,73.809524,10.356164,16.085106,1.758107,-0.033715,-0.262140,0.235205,0.425332,4.501874
Chennai Super Kings,1049,756,34,207,41,68,535,74.928571,22.235294,70.767196,11.117647,18.439024,0.488930,-0.507304,-0.926856,-1.880457,0.425332,3.498005
Mumbai Indians,1131,756,35,245,51,95,506,80.785714,21.600000,66.931217,7.957895,14.823529,-1.111336,1.457839,0.094113,0.562984,0.149668,1.858709
Royal Challengers Bengaluru,1238,802,40,217,68,69,569,83.356608,20.050000,70.947631,11.623188,11.794118,0.564203,-0.821715,0.949579,1.635492,-0.522903,-4.703559
Gujarat Titans,903,648,33,178,28,69,461,75.250000,19.636364,71.141975,9.391304,23.142857,0.645278,0.566359,-2.255157,-1.746366,-0.702386,-7.785864
Delhi Capitals,1035,719,37,228,50,75,484,77.732962,19.432432,67.315716,9.586667,14.380000,-0.950933,0.444858,0.219360,-0.710541,-0.790876,-8.062976
Sunrisers Hyderabad,1245,832,43,246,72,64,578,80.805288,19.348837,69.471154,13.000000,11.555556,-0.051743,-1.677994,1.016946,0.571150,-0.827149,-8.301724


In [195]:
ball_number = df_2024_mid.groupby(['batting_team','ball_no']).agg({'runs_off_bat':'sum','ball':'count'}).reset_index()
ball_number['ball_no'] = ball_number['ball_no'].astype('int')
ball_number = ball_number[ball_number["ball_no"]<=6]

In [196]:
fours_by_ball = ((df_2024_mid[df_2024_mid['runs_off_bat']==4]).groupby(['batting_team','ball_no']).agg(fours=('runs_off_bat','count')).reset_index())
fours_by_ball['ball_no'] = fours_by_ball['ball_no'].astype('int')
fours_by_ball = fours_by_ball[fours_by_ball["ball_no"]<=6]
sixes_by_ball = ((df_2024_mid[df_2024_mid['runs_off_bat']==6]).groupby(['batting_team','ball_no']).agg(sixes=('runs_off_bat','count')).reset_index())
sixes_by_ball['ball_no'] = sixes_by_ball['ball_no'].astype('int')
sixes_by_ball = sixes_by_ball[sixes_by_ball["ball_no"]<=6]
dots_by_ball = ((df_2024_mid[(df_2024_mid['runs_off_bat']==0) & (df_2024_mid['extras']==0)]).groupby(['batting_team','ball_no']).agg(dots=('runs_off_bat','count')).reset_index())
dots_by_ball['ball_no'] = dots_by_ball['ball_no'].astype('int')
dots_by_ball = dots_by_ball[dots_by_ball["ball_no"]<=6]

In [197]:
ball_number = ball_number.merge(fours_by_ball,on=['batting_team','ball_no'])

In [198]:
ball_number = ball_number.merge(sixes_by_ball,on=['batting_team','ball_no'])

In [199]:
ball_number = ball_number.merge(dots_by_ball,on=['batting_team','ball_no'])

In [200]:
ball_number['strike_rate'] = ball_number['runs_off_bat']/ball_number['ball']*100

In [201]:
ball_number_1 = ball_number[ball_number['ball_no']==1]
ball_number_2 = ball_number[ball_number['ball_no']==2]
ball_number_3 = ball_number[ball_number['ball_no']==3]
ball_number_4 = ball_number[ball_number['ball_no']==4]
ball_number_5 = ball_number[ball_number['ball_no']==5]
ball_number_6 = ball_number[ball_number['ball_no']==6]

In [202]:
ball_nos = [ball_number_1,ball_number_2,ball_number_3,ball_number_4,ball_number_5,ball_number_6]

In [203]:
def true_sr(df):
    df['true_strike_rate'] = df['strike_rate'] - (sum(df['runs_off_bat']) / sum(df['ball']) *100)

In [204]:
for ball in ball_nos:
    true_sr(ball)

In [205]:
ball_nos = pd.concat(ball_nos)

In [206]:
ball_number = ball_nos.sort_values(by=['batting_team','ball_no'],inplace=True)

In [207]:
ball_number = ball_nos

In [208]:
ball_number['four_freq'] = ball_number['ball']/ball_number['fours']
ball_number['six_freq'] = ball_number['ball']/ball_number['sixes']
ball_number['dot_freq'] = ball_number['ball']/ball_number['dots']
ball_number

,batting_team,ball_no,runs_off_bat,ball,fours,sixes,dots,strike_rate,true_strike_rate,four_freq,six_freq,dot_freq
0,Chennai Super Kings,1,140,126,9,6,47,111.111111,-18.445149,14.000000,21.000000,2.680851
1,Chennai Super Kings,2,178,126,12,9,39,141.269841,-0.046574,10.500000,14.000000,3.230769
2,Chennai Super Kings,3,176,126,12,10,32,139.682540,1.458908,10.500000,12.600000,3.937500
3,Chennai Super Kings,4,163,126,13,5,34,129.365079,-5.714286,9.692308,25.200000,3.705882
4,Chennai Super Kings,5,160,126,7,7,24,126.984127,-16.611736,18.000000,18.000000,5.250000
5,Chennai Super Kings,6,164,126,13,4,27,130.158730,0.779749,9.692308,31.500000,4.666667
6,Delhi Capitals,1,164,120,15,6,34,136.666667,7.110407,8.000000,20.000000,3.529412
7,Delhi Capitals,2,176,120,14,8,35,146.666667,5.350251,8.571429,15.000000,3.428571
8,Delhi Capitals,3,184,120,8,14,31,153.333333,15.109701,15.000000,8.571429,3.870968
9,Delhi Capitals,4,143,120,11,6,42,119.166667,-15.912698,10.909091,20.000000,2.857143


In [209]:
ball_number[['z_strike_rate', 'z_four_freq', 'z_six_freq', 'z_dot_freq']] = ball_number.groupby('ball_no')[['strike_rate', 'four_freq', 'six_freq', 'dot_freq']].transform(zscore)
ball_number['z_four_freq'] = -ball_number['z_four_freq']
ball_number['z_six_freq'] = -ball_number['z_six_freq']
ball_number_zscore = ball_number.groupby(["batting_team"])[['z_strike_rate', 'z_four_freq', 'z_six_freq', 'z_dot_freq']].sum()
ball_number_zscore['performance_score'] = (0.3 * ball_number_zscore['z_strike_rate'] +
                                        0.25 * ball_number_zscore['z_four_freq'] +
                                        0.25 * ball_number_zscore['z_six_freq'] +
                                        0.2 * ball_number_zscore['z_dot_freq'])
ball_number_zscore.sort_values(by='performance_score',ascending=False)

,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq,performance_score
batting_team,,,,,
Kolkata Knight Riders,5.023036,4.695247,1.987938,-1.934708,2.790765
Royal Challengers Bengaluru,2.715064,-2.870342,3.610741,3.129272,1.625473
Mumbai Indians,2.460642,5.337685,-0.369488,-4.658057,1.048631
Lucknow Super Giants,-0.837692,-0.662781,0.071557,6.240594,0.849005
Sunrisers Hyderabad,1.115092,-5.328365,4.298266,1.309544,0.338911
Delhi Capitals,0.385788,2.392801,0.896047,-3.788325,0.180283
Punjab Kings,-0.802883,-0.452460,2.200052,-4.506862,-0.705339
Rajasthan Royals,-2.884111,-3.599235,-1.031731,1.277517,-1.767471
Chennai Super Kings,-2.979549,-1.630596,-3.222025,1.203546,-1.866311


In [210]:
ball_number[ball_number['ball_no']==1].sort_values(by='true_strike_rate',ascending=False)

,batting_team,ball_no,runs_off_bat,ball,fours,sixes,dots,strike_rate,true_strike_rate,four_freq,six_freq,dot_freq,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq
24,Lucknow Super Giants,1,186,126,15,8,27,147.619048,18.062788,8.400000,15.750000,4.666667,1.706366,0.663094,0.867814,2.204474
48,Royal Challengers Bengaluru,1,186,134,11,10,36,138.805970,9.249710,12.181818,13.400000,3.722222,0.875164,-0.578535,1.439808,0.489811
6,Delhi Capitals,1,164,120,15,6,34,136.666667,7.110407,8.000000,20.000000,3.529412,0.673397,0.794420,-0.166643,0.139759
54,Sunrisers Hyderabad,1,189,139,14,8,36,135.971223,6.414963,9.928571,17.375000,3.861111,0.607807,0.161241,0.472286,0.741968
18,Kolkata Knight Riders,1,163,121,17,5,34,134.710744,5.154484,7.117647,24.200000,3.558824,0.488925,1.084110,-1.188931,0.193157
30,Mumbai Indians,1,159,126,15,6,43,126.190476,-3.365784,8.400000,21.000000,2.930233,-0.314660,0.663094,-0.410045,-0.948066
12,Gujarat Titans,1,135,108,14,4,32,125.000000,-4.556260,7.714286,27.000000,3.375000,-0.426939,0.888224,-1.870455,-0.140580
36,Punjab Kings,1,155,126,11,9,45,123.015873,-6.540387,11.454545,14.000000,2.800000,-0.614071,-0.339760,1.293767,-1.184507
42,Rajasthan Royals,1,158,136,8,7,40,116.176471,-13.379789,17.000000,19.428571,3.400000,-1.259126,-2.160417,-0.027557,-0.095191
0,Chennai Super Kings,1,140,126,9,6,47,111.111111,-18.445149,14.000000,21.000000,2.680851,-1.736863,-1.175471,-0.410045,-1.400825


In [211]:
ball_number[ball_number['ball_no']==2].sort_values(by='true_strike_rate',ascending=False)

,batting_team,ball_no,runs_off_bat,ball,fours,sixes,dots,strike_rate,true_strike_rate,four_freq,six_freq,dot_freq,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq
19,Kolkata Knight Riders,2,184,121,20,8,34,152.066116,10.749700,6.050000,15.125000,3.558824,1.371600,2.086701,-0.056361,-0.635613
7,Delhi Capitals,2,176,120,14,8,35,146.666667,5.350251,8.571429,15.000000,3.428571,0.693778,0.989271,-0.029064,-0.969593
49,Royal Challengers Bengaluru,2,196,134,9,13,31,146.268657,4.952241,14.888889,10.307692,4.322581,0.643814,-1.760350,0.995606,1.322738
37,Punjab Kings,2,184,126,10,12,32,146.031746,4.715330,12.600000,10.500000,3.937500,0.614073,-0.764130,0.953611,0.335352
31,Mumbai Indians,2,183,126,13,10,38,145.238095,3.921680,9.692308,12.600000,3.315789,0.514442,0.501418,0.495030,-1.258777
43,Rajasthan Royals,2,191,135,12,10,31,141.481481,0.165066,11.250000,13.500000,4.354839,0.042854,-0.176554,0.298494,1.405451
1,Chennai Super Kings,2,178,126,12,9,39,141.269841,-0.046574,10.500000,14.000000,3.230769,0.016286,0.149877,0.189308,-1.476778
55,Sunrisers Hyderabad,2,190,139,11,11,36,136.690647,-4.625768,12.636364,12.636364,3.861111,-0.558565,-0.779957,0.487089,0.139483
25,Lucknow Super Giants,2,167,126,11,7,30,132.539683,-8.776733,11.454545,18.000000,4.200000,-1.079657,-0.265581,-0.684181,1.008429
13,Gujarat Titans,2,133,108,10,4,28,123.148148,-18.168267,10.800000,27.000000,3.857143,-2.258627,0.019304,-2.649532,0.129308


In [212]:
ball_number[ball_number['ball_no']==3].sort_values(by='true_strike_rate',ascending=False)

,batting_team,ball_no,runs_off_bat,ball,fours,sixes,dots,strike_rate,true_strike_rate,four_freq,six_freq,dot_freq,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq
44,Rajasthan Royals,3,211,135,15,12,33,156.296296,18.072664,9.000000,11.250000,4.090909,1.221108,0.490390,0.560542,0.545011
8,Delhi Capitals,3,184,120,8,14,31,153.333333,15.109701,15.000000,8.571429,3.870968,1.019670,0.147160,1.071523,0.110237
50,Royal Challengers Bengaluru,3,202,134,11,14,36,150.746269,12.522637,12.181818,9.571429,3.722222,0.843789,0.308374,0.880757,-0.183799
32,Mumbai Indians,3,181,126,16,7,34,143.650794,5.427162,7.875000,18.000000,3.705882,0.361403,0.554745,-0.727128,-0.216099
26,Lucknow Super Giants,3,178,126,8,11,28,141.269841,3.046209,15.750000,11.454545,4.500000,0.199534,0.104256,0.521522,1.353689
2,Chennai Super Kings,3,176,126,12,10,32,139.682540,1.458908,10.500000,12.600000,3.937500,0.091621,0.404582,0.303008,0.241756
20,Kolkata Knight Riders,3,168,121,10,11,35,138.842975,0.619343,12.100000,11.000000,3.457143,0.034543,0.313054,0.608234,-0.707800
14,Gujarat Titans,3,147,108,11,5,23,136.111111,-2.112521,9.818182,21.600000,4.695652,-0.151182,0.443585,-1.413886,1.740449
38,Punjab Kings,3,141,126,9,5,40,111.904762,-26.318870,14.000000,25.200000,3.150000,-1.796851,0.204365,-2.100644,-1.314951
56,Sunrisers Hyderabad,3,155,139,2,11,46,111.510791,-26.712841,69.500000,12.636364,3.021739,-1.823635,-2.970511,0.296071,-1.568493


In [213]:
ball_number[ball_number['ball_no']==4].sort_values(by='true_strike_rate',ascending=False)

,batting_team,ball_no,runs_off_bat,ball,fours,sixes,dots,strike_rate,true_strike_rate,four_freq,six_freq,dot_freq,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq
57,Sunrisers Hyderabad,4,242,139,13,18,33,174.100719,39.021354,10.692308,7.722222,4.212121,2.449295,0.020994,1.597721,1.734964
27,Lucknow Super Giants,4,190,126,15,9,32,150.793651,15.714286,8.400000,14.000000,3.937500,1.001613,0.702184,0.599632,1.208308
21,Kolkata Knight Riders,4,166,120,13,8,37,138.333333,3.253968,9.230769,15.000000,3.243243,0.227661,0.455310,0.440644,-0.123106
51,Royal Challengers Bengaluru,4,182,134,7,13,42,135.820896,0.741530,19.142857,10.307692,3.190476,0.071605,-2.490199,1.186663,-0.224300
15,Gujarat Titans,4,140,108,13,4,31,129.629630,-5.449735,8.307692,27.000000,3.483871,-0.312956,0.729615,-1.467207,0.338359
3,Chennai Super Kings,4,163,126,13,5,34,129.365079,-5.714286,9.692308,25.200000,3.705882,-0.329388,0.318158,-1.181030,0.764123
39,Punjab Kings,4,160,126,11,9,47,126.984127,-8.095238,11.454545,14.000000,2.680851,-0.477277,-0.205515,0.599632,-1.201636
33,Mumbai Indians,4,159,126,20,5,51,126.190476,-8.888889,6.300000,25.200000,2.470588,-0.526573,1.326227,-1.181030,-1.604868
9,Delhi Capitals,4,143,120,11,6,42,119.166667,-15.912698,10.909091,20.000000,2.857143,-0.962846,-0.043426,-0.354294,-0.863552
45,Rajasthan Royals,4,157,135,10,7,41,116.296296,-18.783069,13.500000,19.285714,3.292683,-1.141134,-0.813349,-0.240731,-0.028293


In [214]:
ball_number[ball_number['ball_no']==5].sort_values(by='true_strike_rate',ascending=False)

,batting_team,ball_no,runs_off_bat,ball,fours,sixes,dots,strike_rate,true_strike_rate,four_freq,six_freq,dot_freq,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq
40,Punjab Kings,5,207,126,14,15,40,164.285714,20.689851,9.000000,8.400000,3.150000,1.372965,0.877581,1.002545,-0.940917
34,Mumbai Indians,5,204,126,14,15,43,161.904762,18.308899,9.000000,8.400000,2.930233,1.213872,0.877581,1.002545,-1.197693
22,Kolkata Knight Riders,5,191,119,12,13,29,160.504202,16.908339,9.916667,9.153846,4.103448,1.120288,0.555985,0.871428,0.173091
52,Royal Challengers Bengaluru,5,209,133,12,14,25,157.142857,13.546994,11.083333,9.500000,5.320000,0.895687,0.146681,0.811222,1.594508
16,Gujarat Titans,5,150,108,9,7,26,138.888889,-4.706974,12.000000,15.428571,4.153846,-0.324021,-0.174915,-0.219936,0.231976
10,Delhi Capitals,5,166,120,14,7,32,138.333333,-5.262530,8.571429,17.142857,3.750000,-0.361142,1.027937,-0.518102,-0.239878
58,Sunrisers Hyderabad,5,189,138,10,13,47,136.956522,-6.639341,13.800000,10.615385,2.936170,-0.453139,-0.806412,0.617223,-1.190755
28,Lucknow Super Giants,5,164,126,9,7,27,130.158730,-13.437133,14.000000,18.000000,4.666667,-0.907360,-0.876578,-0.667185,0.831154
4,Chennai Super Kings,5,160,126,7,7,24,126.984127,-16.611736,18.000000,18.000000,5.250000,-1.119483,-2.279905,-0.667185,1.512720
46,Rajasthan Royals,5,165,135,14,5,41,122.222222,-21.373641,9.642857,27.000000,3.292683,-1.437668,0.652046,-2.232556,-0.774206


In [215]:
ball_number[ball_number['ball_no']==6].sort_values(by='true_strike_rate',ascending=False)

,batting_team,ball_no,runs_off_bat,ball,fours,sixes,dots,strike_rate,true_strike_rate,four_freq,six_freq,dot_freq,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq
23,Kolkata Knight Riders,6,172,119,11,10,37,144.537815,15.158834,10.818182,11.900000,3.216216,1.780018,0.200087,1.312923,-0.834436
35,Mumbai Indians,6,176,126,14,7,31,139.682540,10.303559,9.000000,18.000000,4.064516,1.212158,1.414620,0.451141,0.567447
59,Sunrisers Hyderabad,6,189,138,11,9,30,136.956522,7.577541,12.545455,15.333333,4.600000,0.893330,-0.953720,0.827876,1.452377
5,Chennai Super Kings,6,164,126,13,4,27,130.158730,0.779749,9.692308,31.500000,4.666667,0.098278,0.952163,-1.456082,1.562549
41,Punjab Kings,6,164,126,11,7,35,130.158730,0.779749,11.454545,18.000000,3.600000,0.098278,-0.225000,0.451141,-0.200203
47,Rajasthan Royals,6,171,135,10,8,35,126.666667,-2.712314,13.500000,16.875000,3.857143,-0.310145,-1.591351,0.610076,0.224746
53,Royal Challengers Bengaluru,6,165,133,15,4,35,124.060150,-5.318831,8.866667,33.250000,3.800000,-0.614996,1.503686,-1.703315,0.130313
11,Delhi Capitals,6,147,119,10,8,47,123.529412,-5.849569,11.900000,14.875000,2.531915,-0.677070,-0.522561,0.892627,-1.965299
17,Gujarat Titans,6,133,108,10,4,32,123.148148,-6.230833,10.800000,27.000000,3.375000,-0.721661,0.212232,-0.820341,-0.572034
29,Lucknow Super Giants,6,144,126,10,5,36,114.285714,-15.093267,12.600000,25.200000,3.500000,-1.758188,-0.990156,-0.566045,-0.365461


In [216]:
ball_number[ball_number['batting_team']=='Kolkata Knight Riders']
#kkr is the only team with all +ve true strike rates, which means it has consistency in attack, never slows down 

,batting_team,ball_no,runs_off_bat,ball,fours,sixes,dots,strike_rate,true_strike_rate,four_freq,six_freq,dot_freq,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq
18,Kolkata Knight Riders,1,163,121,17,5,34,134.710744,5.154484,7.117647,24.200000,3.558824,0.488925,1.084110,-1.188931,0.193157
19,Kolkata Knight Riders,2,184,121,20,8,34,152.066116,10.749700,6.050000,15.125000,3.558824,1.371600,2.086701,-0.056361,-0.635613
20,Kolkata Knight Riders,3,168,121,10,11,35,138.842975,0.619343,12.100000,11.000000,3.457143,0.034543,0.313054,0.608234,-0.707800
21,Kolkata Knight Riders,4,166,120,13,8,37,138.333333,3.253968,9.230769,15.000000,3.243243,0.227661,0.455310,0.440644,-0.123106
22,Kolkata Knight Riders,5,191,119,12,13,29,160.504202,16.908339,9.916667,9.153846,4.103448,1.120288,0.555985,0.871428,0.173091
23,Kolkata Knight Riders,6,172,119,11,10,37,144.537815,15.158834,10.818182,11.900000,3.216216,1.780018,0.200087,1.312923,-0.834436


In [217]:
ball_number_zscore.sort_values(by='performance_score',ascending=False)

,z_strike_rate,z_four_freq,z_six_freq,z_dot_freq,performance_score
batting_team,,,,,
Kolkata Knight Riders,5.023036,4.695247,1.987938,-1.934708,2.790765
Royal Challengers Bengaluru,2.715064,-2.870342,3.610741,3.129272,1.625473
Mumbai Indians,2.460642,5.337685,-0.369488,-4.658057,1.048631
Lucknow Super Giants,-0.837692,-0.662781,0.071557,6.240594,0.849005
Sunrisers Hyderabad,1.115092,-5.328365,4.298266,1.309544,0.338911
Delhi Capitals,0.385788,2.392801,0.896047,-3.788325,0.180283
Punjab Kings,-0.802883,-0.452460,2.200052,-4.506862,-0.705339
Rajasthan Royals,-2.884111,-3.599235,-1.031731,1.277517,-1.767471
Chennai Super Kings,-2.979549,-1.630596,-3.222025,1.203546,-1.866311


In [218]:
team_mid_2024_bat

,runs,balls,player_dismissed,dots,sixes,fours,activity,avg_score,bpd,activity_pct,four_frequency,six_frequency,z_score_activity_pct,z_score_four_frequency,z_score_six_frequency,z_score_avg_score,z_score_bpd,z_performance_index
batting_team,,,,,,,,,,,,,,,,,,
Chennai Super Kings,1049,756,34,207,41,68,535,74.928571,22.235294,70.767196,11.117647,18.439024,0.488930,-0.507304,-0.926856,-1.880457,0.425332,3.498005
Delhi Capitals,1035,719,37,228,50,75,484,77.732962,19.432432,67.315716,9.586667,14.380000,-0.950933,0.444858,0.219360,-0.710541,-0.790876,-8.062976
Gujarat Titans,903,648,33,178,28,69,461,75.250000,19.636364,71.141975,9.391304,23.142857,0.645278,0.566359,-2.255157,-1.746366,-0.702386,-7.785864
Kolkata Knight Riders,1175,721,28,213,58,91,499,88.002774,25.750000,69.209431,7.923077,12.431034,-0.160927,1.479493,0.769722,3.573746,1.950420,20.935805
Lucknow Super Giants,1120,756,34,193,47,73,558,80.000000,22.235294,73.809524,10.356164,16.085106,1.758107,-0.033715,-0.262140,0.235205,0.425332,4.501874
Mumbai Indians,1131,756,35,245,51,95,506,80.785714,21.600000,66.931217,7.957895,14.823529,-1.111336,1.457839,0.094113,0.562984,0.149668,1.858709
Punjab Kings,1071,756,41,247,58,68,497,76.500000,18.439024,65.740741,11.117647,13.034483,-1.607971,-0.507304,0.599317,-1.224900,-1.221932,-12.743733
Rajasthan Royals,1155,810,34,231,51,74,572,77.000000,23.823529,70.617284,10.945946,15.882353,0.426391,-0.400518,-0.204885,-1.016313,1.114493,10.803462
Royal Challengers Bengaluru,1238,802,40,217,68,69,569,83.356608,20.050000,70.947631,11.623188,11.794118,0.564203,-0.821715,0.949579,1.635492,-0.522903,-4.703559


In [219]:
team_mid_2024_bat_true = pd.DataFrame()
team_mid_2024_bat_true['runs'] = team_mid_2024_bat['runs']
team_mid_2024_bat_true['balls'] = team_mid_2024_bat['balls']
team_mid_2024_bat_true['player_dismissed'] = team_mid_2024_bat['player_dismissed']
team_mid_2024_bat_true['true_avg_score'] = -sum(team_mid_2024_bat_true['runs'])/sum(team_mid_2024_bat_true['balls'])*54 + team_mid_2024_bat['avg_score']
team_mid_2024_bat_true['avg_score_rank'] = team_mid_2024_bat_true['true_avg_score'].rank(ascending=False).astype('int')
team_mid_2024_bat_true['true_bpd'] = -sum(team_mid_2024_bat_true['balls'])/sum(team_mid_2024_bat_true['player_dismissed']) + team_mid_2024_bat['bpd']
team_mid_2024_bat_true['bpd_rank'] = team_mid_2024_bat_true['true_bpd'].rank(ascending=False).astype('int')
team_mid_2024_bat_true['true_six_freq'] = sum(team_mid_2024_bat['balls'])/sum(team_mid_2024_bat['sixes']) - team_mid_2024_bat['six_frequency']
team_mid_2024_bat_true['six_freq_rank'] = team_mid_2024_bat_true['true_six_freq'].rank(ascending=False).astype('int')
team_mid_2024_bat_true['true_four_freq'] = sum(team_mid_2024_bat['balls'])/sum(team_mid_2024_bat['fours']) - team_mid_2024_bat['four_frequency']
team_mid_2024_bat_true['four_freq_rank'] = team_mid_2024_bat_true['true_four_freq'].rank(ascending=False).astype('int')
team_mid_2024_bat_true['true_activity_rate'] = -sum(team_mid_2024_bat['activity'])/sum(team_mid_2024_bat['balls'])*100 + team_mid_2024_bat['activity_pct']
team_mid_2024_bat_true['activity_rank'] = team_mid_2024_bat_true['true_activity_rate'].rank(ascending=False).astype('int')
team_mid_2024_bat_true['z_score_rank'] = team_mid_2024_bat['z_performance_index'].rank(ascending=False).astype('int')

In [220]:
team_mid_2024_bat

,runs,balls,player_dismissed,dots,sixes,fours,activity,avg_score,bpd,activity_pct,four_frequency,six_frequency,z_score_activity_pct,z_score_four_frequency,z_score_six_frequency,z_score_avg_score,z_score_bpd,z_performance_index
batting_team,,,,,,,,,,,,,,,,,,
Chennai Super Kings,1049,756,34,207,41,68,535,74.928571,22.235294,70.767196,11.117647,18.439024,0.488930,-0.507304,-0.926856,-1.880457,0.425332,3.498005
Delhi Capitals,1035,719,37,228,50,75,484,77.732962,19.432432,67.315716,9.586667,14.380000,-0.950933,0.444858,0.219360,-0.710541,-0.790876,-8.062976
Gujarat Titans,903,648,33,178,28,69,461,75.250000,19.636364,71.141975,9.391304,23.142857,0.645278,0.566359,-2.255157,-1.746366,-0.702386,-7.785864
Kolkata Knight Riders,1175,721,28,213,58,91,499,88.002774,25.750000,69.209431,7.923077,12.431034,-0.160927,1.479493,0.769722,3.573746,1.950420,20.935805
Lucknow Super Giants,1120,756,34,193,47,73,558,80.000000,22.235294,73.809524,10.356164,16.085106,1.758107,-0.033715,-0.262140,0.235205,0.425332,4.501874
Mumbai Indians,1131,756,35,245,51,95,506,80.785714,21.600000,66.931217,7.957895,14.823529,-1.111336,1.457839,0.094113,0.562984,0.149668,1.858709
Punjab Kings,1071,756,41,247,58,68,497,76.500000,18.439024,65.740741,11.117647,13.034483,-1.607971,-0.507304,0.599317,-1.224900,-1.221932,-12.743733
Rajasthan Royals,1155,810,34,231,51,74,572,77.000000,23.823529,70.617284,10.945946,15.882353,0.426391,-0.400518,-0.204885,-1.016313,1.114493,10.803462
Royal Challengers Bengaluru,1238,802,40,217,68,69,569,83.356608,20.050000,70.947631,11.623188,11.794118,0.564203,-0.821715,0.949579,1.635492,-0.522903,-4.703559


In [221]:
team_mid_2024_bat_true

,runs,balls,player_dismissed,true_avg_score,avg_score_rank,true_bpd,bpd_rank,true_six_freq,six_freq_rank,true_four_freq,four_freq_rank,true_activity_rate,activity_rank,z_score_rank
batting_team,,,,,,,,,,,,,,
Chennai Super Kings,1049,756,34,-4.556341,10,1.187940,3,-4.019177,9,-0.988961,7,1.166878,4,4
Delhi Capitals,1035,719,37,-1.751950,6,-1.614921,8,0.039847,5,0.542020,4,-2.284601,8,8
Gujarat Titans,903,648,33,-4.234913,9,-1.410990,7,-8.723010,10,0.737382,3,1.541658,2,7
Kolkata Knight Riders,1175,721,28,8.517861,1,4.702646,1,1.988813,3,2.205609,1,-0.390886,7,1
Lucknow Super Giants,1120,756,34,0.515087,5,1.187940,3,-1.665259,8,-0.227478,5,4.209206,1,3
Mumbai Indians,1131,756,35,1.300802,4,0.552646,5,-0.403682,6,2.170792,2,-2.669101,9,5
Punjab Kings,1071,756,41,-2.984913,8,-2.608329,10,1.385365,4,-0.988961,7,-3.859577,10,10
Rajasthan Royals,1155,810,34,-2.484913,7,2.776176,2,-1.462506,7,-0.817260,6,1.016966,5,2
Royal Challengers Bengaluru,1238,802,40,3.871696,2,-0.997354,6,2.625730,2,-1.494502,9,1.347313,3,6


# Middle Overs : Player Batting

In [222]:
mid_batters = df_2024_mid.groupby(['striker']).agg({'batting_team':'first','runs_off_bat':'sum','extras':'sum',
                                                    'ball':'count','wides':'count','player_dismissed':'count'})

In [223]:
mid_batters['runs'] = mid_batters['runs_off_bat'] - mid_batters['extras']
mid_batters['balls'] = mid_batters['ball'] - mid_batters['wides']
mid_batters['dismissals'] = mid_batters['player_dismissed']
mid_batters.drop(columns={'runs_off_bat','ball','extras','wides','player_dismissed'},inplace=True)

In [224]:
mid_batters['strike_rate'] = mid_batters['runs']/mid_batters['balls']*100
mid_batters['bpd'] = mid_batters['balls']/mid_batters['dismissals']

In [225]:
mid_batters['fours'] = df_2024_mid[df_2024_mid['runs_off_bat']==4].groupby(['striker']).agg({'runs_off_bat':'count'})
mid_batters['sixes'] = df_2024_mid[df_2024_mid['runs_off_bat']==6].groupby(['striker']).agg({'runs_off_bat':'count'})
#mid_batters['dots'] = df_2024_mid[(df_2024_mid['runs_off_bat']==0) & (df_2024_mid['extras']==0)].groupby(['striker']).agg({'runs_off_bat':'count'})
mid_batters['activity'] = df_2024_mid[(df_2024_mid['runs_off_bat']!=0) & (df_2024_mid['extras']==0)].groupby(['striker']).agg({'runs_off_bat':'count'})
mid_batters['fours'] = mid_batters['fours'].fillna(0).astype('int')
mid_batters['sixes'] = mid_batters['sixes'].fillna(0).astype('int')
mid_batters['activity'] = mid_batters['activity'].fillna(0).astype('int')
#mid_batters['dots'] = mid_batters['dots'].fillna(0).astype('int')

In [226]:
mid_batters['fours_freq'] = mid_batters['balls']/mid_batters['fours']
mid_batters['sixes_freq'] = mid_batters['balls']/mid_batters['sixes']
mid_batters['activity_rate'] = mid_batters['activity']/mid_batters['balls']*100

In [227]:
mid_batters['true_strike_rate'] = -sum(mid_batters['runs'])/sum(mid_batters['balls'])*100 + mid_batters['strike_rate']
mid_batters['true_fours_freq'] = sum(mid_batters['balls'])/sum(mid_batters['fours']) - mid_batters['fours_freq']
mid_batters['true_sixes_freq'] = sum(mid_batters['balls'])/sum(mid_batters['sixes']) - mid_batters['sixes_freq']
mid_batters['true_activity_rate']= -sum(mid_batters['activity'])/sum(mid_batters['balls'])*100+mid_batters['activity_rate']
mid_batters['true_bpd'] = -sum(mid_batters['balls'])/sum(mid_batters['dismissals']) + mid_batters['bpd'] 

In [228]:
mid_batters

,batting_team,runs,balls,dismissals,strike_rate,bpd,fours,sixes,activity,fours_freq,sixes_freq,activity_rate,true_strike_rate,true_fours_freq,true_sixes_freq,true_activity_rate,true_bpd
striker,,,,,,,,,,,,,,,,,
A Badoni,Lucknow Super Giants,85,71,3,119.718310,23.666667,7,2,54,10.142857,35.500000,76.056338,-15.031031,0.018001,-21.034351,6.676391,2.552461
A Manohar,Gujarat Titans,9,10,2,90.000000,5.000000,1,0,5,10.000000,inf,50.000000,-44.749340,0.160858,-inf,-19.379947,-16.114206
A Raghuvanshi,Kolkata Knight Riders,81,55,3,147.272727,18.333333,7,4,43,7.857143,13.750000,78.181818,12.523387,2.303715,0.715649,8.801871,-2.780873
AD Russell,Kolkata Knight Riders,77,56,2,137.500000,28.000000,9,5,33,6.222222,11.200000,58.928571,2.750660,3.938636,3.265649,-10.451376,6.885794
AJ Turner,Lucknow Super Giants,14,10,1,140.000000,10.000000,0,2,7,inf,5.000000,70.000000,5.250660,-inf,9.465649,0.620053,-11.114206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VR Iyer,Kolkata Knight Riders,192,140,3,137.142857,46.666667,14,10,92,10.000000,14.000000,65.714286,2.393517,0.160858,0.465649,-3.665662,25.552461
WG Jacks,Royal Challengers Bengaluru,121,77,4,157.142857,19.250000,9,9,52,8.555556,8.555556,67.532468,22.393517,1.605302,5.910093,-1.847480,-1.864206
WP Saha,Gujarat Titans,5,10,1,50.000000,10.000000,0,0,5,inf,inf,50.000000,-84.749340,-inf,-inf,-19.379947,-11.114206


In [229]:
mid_batters['balls'].describe()

count    129.000000
mean      58.759690
std       56.927829
min        1.000000
25%       11.000000
50%       39.000000
75%       84.000000
max      268.000000
Name: balls, dtype: float64

In [230]:
mid_batters['balls'].quantile(0.55)

49.60000000000002

In [231]:
mid_batters_filter = mid_batters[mid_batters['balls']>=60]

In [232]:
mid_batters_filter

,batting_team,runs,balls,dismissals,strike_rate,bpd,fours,sixes,activity,fours_freq,sixes_freq,activity_rate,true_strike_rate,true_fours_freq,true_sixes_freq,true_activity_rate,true_bpd
striker,,,,,,,,,,,,,,,,,
A Badoni,Lucknow Super Giants,85,71,3,119.718310,23.666667,7,2,54,10.142857,35.500000,76.056338,-15.031031,0.018001,-21.034351,6.676391,2.552461
AK Markram,Sunrisers Hyderabad,123,111,6,110.810811,18.500000,6,2,83,18.500000,55.500000,74.774775,-23.938530,-8.339142,-41.034351,5.394828,-2.614206
AM Rahane,Chennai Super Kings,93,82,7,113.414634,11.714286,7,2,59,11.714286,41.000000,71.951220,-21.334706,-1.553428,-26.534351,2.571272,-9.399920
AR Patel,Delhi Capitals,126,93,4,135.483871,23.250000,12,6,57,7.750000,15.500000,61.290323,0.734531,2.410858,-1.034351,-8.089625,2.135794
Abhishek Sharma,Sunrisers Hyderabad,132,68,6,194.117647,11.333333,9,13,47,7.555556,5.230769,69.117647,59.368307,2.605302,9.234880,-0.262300,-9.780873
Abishek Porel,Delhi Capitals,143,97,6,147.422680,16.166667,14,4,70,6.928571,24.250000,72.164948,12.673340,3.232286,-9.784351,2.785001,-4.947539
B Sai Sudharsan,Gujarat Titans,299,205,6,145.853659,34.166667,29,9,163,7.068966,22.777778,79.512195,11.104318,3.091892,-8.312129,10.132248,13.052461
C Green,Royal Challengers Bengaluru,103,96,6,107.291667,16.000000,8,3,65,12.000000,32.000000,67.708333,-27.457674,-1.839142,-17.534351,-1.671614,-5.114206
DA Miller,Gujarat Titans,88,76,3,115.789474,25.333333,9,2,54,8.444444,38.000000,71.052632,-18.959867,1.716413,-23.534351,1.672684,4.219127


In [233]:
mid_batters_filter[['batting_team','runs','balls','dismissals','strike_rate','true_strike_rate']].sort_values(by='strike_rate',ascending=False).reset_index()

,striker,batting_team,runs,balls,dismissals,strike_rate,true_strike_rate
0,Abhishek Sharma,Sunrisers Hyderabad,132,68,6,194.117647,59.368307
1,SP Narine,Kolkata Knight Riders,161,93,7,173.118280,38.368939
2,F du Plessis,Royal Challengers Bengaluru,126,73,3,172.602740,37.853399
3,RR Rossouw,Punjab Kings,108,63,4,171.428571,36.679231
4,RM Patidar,Royal Challengers Bengaluru,294,172,11,170.930233,36.180892
5,PD Salt,Kolkata Knight Riders,128,76,4,168.421053,33.671712
6,N Pooran,Lucknow Super Giants,238,143,3,166.433566,31.684226
7,JM Bairstow,Punjab Kings,118,72,4,163.888889,29.139549
8,SA Yadav,Mumbai Indians,219,134,5,163.432836,28.683495
9,WG Jacks,Royal Challengers Bengaluru,121,77,4,157.142857,22.393517


In [234]:
mid_batters_filter[['batting_team','runs','balls','fours','fours_freq','true_fours_freq']].sort_values(by='fours_freq',ascending=True).reset_index()

,striker,batting_team,runs,balls,fours,fours_freq,true_fours_freq
0,PD Salt,Kolkata Knight Riders,128,76,13,5.846154,4.314704
1,RG Sharma,Mumbai Indians,118,87,14,6.214286,3.946572
2,SA Yadav,Mumbai Indians,219,134,21,6.380952,3.779906
3,Tilak Varma,Mumbai Indians,291,192,29,6.620690,3.540168
4,SS Iyer,Kolkata Knight Riders,211,157,23,6.826087,3.334771
5,Abishek Porel,Delhi Capitals,143,97,14,6.928571,3.232286
6,B Sai Sudharsan,Gujarat Titans,299,205,29,7.068966,3.091892
7,SP Narine,Kolkata Knight Riders,161,93,13,7.153846,3.007012
8,Abhishek Sharma,Sunrisers Hyderabad,132,68,9,7.555556,2.605302
9,AR Patel,Delhi Capitals,126,93,12,7.750000,2.410858


In [235]:
mid_batters_filter[['batting_team','runs','balls','sixes','sixes_freq','true_sixes_freq']].sort_values(by='sixes_freq',ascending=True).reset_index()

,striker,batting_team,runs,balls,sixes,sixes_freq,true_sixes_freq
0,Abhishek Sharma,Sunrisers Hyderabad,132,68,13,5.230769,9.234880
1,RM Patidar,Royal Challengers Bengaluru,294,172,30,5.733333,8.732316
2,SP Narine,Kolkata Knight Riders,161,93,14,6.642857,7.822792
3,RR Rossouw,Punjab Kings,108,63,9,7.000000,7.465649
4,N Pooran,Lucknow Super Giants,238,143,18,7.944444,6.521204
5,JM Bairstow,Punjab Kings,118,72,9,8.000000,6.465649
6,WG Jacks,Royal Challengers Bengaluru,121,77,9,8.555556,5.910093
7,H Klaasen,Sunrisers Hyderabad,259,181,21,8.619048,5.846601
8,P Simran Singh,Punjab Kings,91,65,7,9.285714,5.179935
9,Nithish Kumar Reddy,Sunrisers Hyderabad,207,142,14,10.142857,4.322792


In [236]:
mid_batters_filter[['batting_team','runs','balls','activity','activity_rate','true_activity_rate']].sort_values(by='activity_rate',ascending=False).reset_index()

,striker,batting_team,runs,balls,activity,activity_rate,true_activity_rate
0,F du Plessis,Royal Challengers Bengaluru,126,73,65,89.041096,19.661149
1,Shubman Gill,Gujarat Titans,173,112,92,82.142857,12.762910
2,B Sai Sudharsan,Gujarat Titans,299,205,163,79.512195,10.132248
3,KL Rahul,Lucknow Super Giants,217,156,124,79.487179,10.107232
4,SV Samson,Rajasthan Royals,269,193,150,77.720207,8.340260
5,N Pooran,Lucknow Super Giants,238,143,109,76.223776,6.843829
6,YBK Jaiswal,Rajasthan Royals,131,84,64,76.190476,6.810529
7,A Badoni,Lucknow Super Giants,85,71,54,76.056338,6.676391
8,DJ Mitchell,Chennai Super Kings,172,112,85,75.892857,6.512910
9,RD Gaikwad,Chennai Super Kings,230,182,138,75.824176,6.444229


In [237]:
mid_batters[mid_batters["batting_team"]=='Kolkata Knight Riders'][['runs','balls','dismissals','strike_rate','true_strike_rate','fours_freq','true_fours_freq','sixes_freq','true_sixes_freq','activity_rate','true_activity_rate']]

,runs,balls,dismissals,strike_rate,true_strike_rate,fours_freq,true_fours_freq,sixes_freq,true_sixes_freq,activity_rate,true_activity_rate
striker,,,,,,,,,,,
A Raghuvanshi,81,55,3,147.272727,12.523387,7.857143,2.303715,13.750000,0.715649,78.181818,8.801871
AD Russell,77,56,2,137.500000,2.750660,6.222222,3.938636,11.200000,3.265649,58.928571,-10.451376
MK Pandey,33,26,0,126.923077,-7.826263,13.000000,-2.839142,26.000000,-11.534351,76.923077,7.543130
N Rana,29,20,2,145.000000,10.250660,6.666667,3.494191,20.000000,-5.534351,75.000000,5.620053
PD Salt,128,76,4,168.421053,33.671712,5.846154,4.314704,10.857143,3.608506,71.052632,1.672684
RK Singh,72,58,3,124.137931,-10.611409,14.500000,-4.339142,19.333333,-4.867684,67.241379,-2.138568
Rahmanullah Gurbaz,14,10,1,140.000000,5.250660,10.000000,0.160858,5.000000,9.465649,50.000000,-19.379947
Ramandeep Singh,55,34,2,161.764706,27.015366,17.000000,-6.839142,6.800000,7.665649,64.705882,-4.674065
SP Narine,161,93,7,173.118280,38.368939,7.153846,3.007012,6.642857,7.822792,68.817204,-0.562743


In [238]:
#middle overs: team performance: kkr second to none, except activity rate and all top teams had poor activity rates, 
#showing high six/four rate can compensate low activity rate. 
#indv performance: no specialist middle order player for kkr with high true_sr, only narine and salt feature in top 7
#and their entry points are inside the powerplay, however, players like Raghuvanshi, V Iyer, Russell, Ramandeep
#S Iyer all had positive true_srs,  showing that the cumulative effort was greater than the sum of its parts

In [239]:
#Main player to be analysed: Venkatesh Iyer - every metric +ve except activity and everything just +ve, also he gets out 
#very seldom

#V Iyer's entry and exit points; striker and non striker - runs division, non striker percentage division

In [240]:
v_iyer = df_2024[(df_2024["striker"]=='VR Iyer') | (df_2024["non_striker"]=='VR Iyer')]

In [241]:
v_iyer_by_match = v_iyer[v_iyer["striker"]=='VR Iyer'].groupby(["match_id"]).agg(entry_point=('ball', 'min'), 
exit_point=('ball', 'max'),runs=('runs_off_bat','sum'),balls=('ball','count'),wides=('wides','count'))
v_iyer_by_match["balls"] = v_iyer_by_match["balls"] - v_iyer_by_match["wides"]
v_iyer_by_match.drop(columns={'wides'},inplace=True)

In [242]:
v_iyer_ns = v_iyer[v_iyer["non_striker"]=='VR Iyer'].groupby(["match_id"]).agg({'runs_off_bat':'sum','ball':'count',
'wides':'count'})
v_iyer_ns["ball"] = v_iyer_ns["ball"] - v_iyer_ns["wides"]
v_iyer_ns.drop(columns={'wides'},inplace=True)
v_iyer_ns["strike_rate"] = v_iyer_ns["runs_off_bat"]/v_iyer_ns["ball"]*100
v_iyer_by_match["non_striker_runs"]  = v_iyer_ns["runs_off_bat"]
v_iyer_by_match["non_striker_balls"] = v_iyer_ns["ball"]
v_iyer_by_match["non_striker_strike_rate"] = v_iyer_ns["strike_rate"]

In [243]:
v_iyer_by_match["strike_rate"] = v_iyer_by_match["runs"]/v_iyer_by_match["balls"]*100
sum(v_iyer_by_match["non_striker_balls"])

239

In [244]:
v_iyer_non_strikers=v_iyer.groupby(["striker"]).agg({'runs_off_bat':'sum','ball':'count','wides':'count'})
v_iyer_non_strikers['ball'] = v_iyer_non_strikers['ball'] - v_iyer_non_strikers["wides"] 
v_iyer_non_strikers.drop(columns={'wides'},inplace=True)
v_iyer_non_strikers["strike_rate"] = v_iyer_non_strikers["runs_off_bat"]/v_iyer_non_strikers["ball"]*100

In [245]:
v_iyer_non_strikers

,runs_off_bat,ball,strike_rate
striker,,,
A Raghuvanshi,0,1,0.000000
AD Russell,31,15,206.666667
MA Starc,1,3,33.333333
MK Pandey,42,31,135.483871
N Rana,18,12,150.000000
PD Salt,18,13,138.461538
RK Singh,26,17,152.941176
Rahmanullah Gurbaz,35,26,134.615385
Ramandeep Singh,20,11,181.818182


In [246]:
v_iyer_by_match["flag"] = v_iyer_by_match.apply(
    lambda row: np.nan if row["balls"] <= 10 
    else (1 if row['balls'] >=10 and row["strike_rate"] >= row["non_striker_strike_rate"] else 0),
    axis=1
)


In [247]:
v_iyer_by_match

,entry_point,exit_point,runs,balls,non_striker_runs,non_striker_balls,non_striker_strike_rate,strike_rate,flag
match_id,,,,,,,,,
1422121,2.1,3.3,7,5,2,4,50.000000,140.000000,NaN
1422128,6.4,15.1,50,30,27,22,122.727273,166.666667,1.0
1422134,19.5,19.6,5,2,3,4,75.000000,250.000000,NaN
1426260,6.7,8.2,3,8,1,1,100.000000,37.500000,NaN
1426269,17.4,19.2,8,6,12,5,240.000000,133.333333,NaN
1426274,5.3,8.2,16,8,15,10,150.000000,200.000000,NaN
1426280,10.3,19.7,39,23,78,35,222.857143,169.565217,0.0
1426285,9.3,16.3,26,23,31,20,155.000000,113.043478,0.0
1426289,3.2,19.5,70,52,67,50,134.000000,134.615385,1.0


In [248]:
#clearly, Iyer either got out cheaply or made it count in  most cases,
#in cases when he batted for atleast 10 balls, he batted faster than his partners 4/7 times
#for 3/7 times when he batted slower, the non strikers were either too fast or the chase was a calkwalk by then
#also Shreyas Iyer's strike rate shot up when he was batting with Venkatesh Iyer

In [249]:
df_2024[(df_2024["striker"]=='SS Iyer') & (df_2024["non_striker"]!='VR Iyer')].groupby(['striker']).agg({'runs_off_bat':'sum','ball':'count'})

,runs_off_bat,ball
striker,,
SS Iyer,179,153


In [250]:
179/153*100

116.99346405228759

In [251]:
df_2024[(df_2024["striker"]=='SS Iyer') & (df_2024["non_striker"]=='VR Iyer')].groupby(['striker']).agg({'runs_off_bat':'sum','ball':'count'})

,runs_off_bat,ball
striker,,
SS Iyer,172,97


In [252]:
172/97*100

177.31958762886597

In [253]:
v_iyer[v_iyer["striker"]=='VR Iyer'].groupby(['bowler']).agg({'runs_off_bat':'sum',
'ball':'count'}).sort_values(by='runs_off_bat',ascending=False).head(15)

,runs_off_bat,ball
bowler,,
B Kumar,30,14
T Natarajan,29,12
G Coetzee,21,9
HH Pandya,21,13
AS Joseph,20,6
N Thushara,17,12
JJ Bumrah,17,11
V Viyaskanth,15,5
A Kamboj,14,8


In [254]:
df_2024[(df_2024["striker"]=='SS Iyer')].groupby(["bowler"]).agg({'runs_off_bat':'sum',
'ball':'count'}).sort_values(by='runs_off_bat',ascending=False).head(15)

,runs_off_bat,ball
bowler,,
TM Head,27,6
SM Curran,23,5
KV Sharma,19,15
AR Patel,17,12
Yash Dayal,16,7
S Joseph,16,12
Vijaykumar Vyshak,15,13
Yash Thakur,15,14
Kuldeep Yadav,12,7


# Death Overs: Team Batting

In [255]:
df_2024_death = df_2024[df_2024["ball"]>=15]

In [256]:
df_2024_death

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,current_score,player_dismissed_flag,current_wickets,over_no,ball_no
243909,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,15.1,Royal Challengers Bengaluru,Chennai Super Kings,KD Karthik,Anuj Rawat,M Theekshana,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106,0,5,15,1
243910,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,15.2,Royal Challengers Bengaluru,Chennai Super Kings,KD Karthik,Anuj Rawat,M Theekshana,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107,0,5,15,2
243911,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,15.3,Royal Challengers Bengaluru,Chennai Super Kings,Anuj Rawat,KD Karthik,M Theekshana,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,0,5,15,3
243912,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,15.4,Royal Challengers Bengaluru,Chennai Super Kings,KD Karthik,Anuj Rawat,M Theekshana,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,0,5,15,4
243913,1422119,2024,2024-03-22,"MA Chidambaram Stadium, Chepauk, Chennai",1,15.5,Royal Challengers Bengaluru,Chennai Super Kings,Anuj Rawat,KD Karthik,M Theekshana,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110,0,5,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260848,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",1,17.5,Sunrisers Hyderabad,Kolkata Knight Riders,JD Unadkat,PJ Cummins,SP Narine,0,0,NaN,NaN,NaN,NaN,NaN,lbw,JD Unadkat,NaN,NaN,113,1,9,17,5
260849,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",1,17.6,Sunrisers Hyderabad,Kolkata Knight Riders,B Kumar,PJ Cummins,SP Narine,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113,0,9,17,6
260850,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",1,18.1,Sunrisers Hyderabad,Kolkata Knight Riders,PJ Cummins,B Kumar,AD Russell,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113,0,9,18,1
260851,1426312,2024,2024-05-26,"MA Chidambaram Stadium, Chepauk, Chennai",1,18.2,Sunrisers Hyderabad,Kolkata Knight Riders,PJ Cummins,B Kumar,AD Russell,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113,0,9,18,2


In [257]:
#metrics: team: sr/run-rate,bpd , six hitting, four hitting, entry point: runs/wickets, 15+/20+ overs --done
#player: 5 ball sr, 10 ball sr, six/four hitting, true strike rate, consecutive boundaries --done
#misc: while chasing: maintainence of run rate
#player case studies: Raman/Russell

In [258]:
death_team = df_2024_death.groupby(["batting_team"]).agg(runs=('runs_off_bat','sum'),extras=('extras','sum'),
balls=('ball','count'),wides=('wides','count'),noballs=('noballs','count'),
dismissals=('player_dismissed','count'))

In [259]:
death_team["runs"] = death_team['runs'] + death_team['extras']
death_team['balls'] = death_team['balls'] - death_team['wides'] - death_team['noballs']
death_team.drop(columns={'extras','wides','noballs'},inplace=True)

In [260]:
death_team['avg_score'] = death_team['runs']/death_team['balls']*30
death_team['bpd'] = death_team['balls']/death_team['dismissals']

In [261]:
death_team = death_team.merge(df_2024_death[df_2024_death["runs_off_bat"]==6].groupby(["batting_team"]).agg(sixes=('runs_off_bat','count')),on='batting_team')
death_team = death_team.merge(df_2024_death[df_2024_death["runs_off_bat"]==4].groupby(["batting_team"]).agg(fours=('runs_off_bat','count')),on='batting_team')


In [262]:
death_team['six_frequency'] = death_team["balls"]/death_team["sixes"]
death_team['four_frequency'] = death_team['balls']/death_team["fours"]

In [263]:
death_team['true_avg_score'] = -sum(death_team['runs'])/sum(death_team['balls'])*30 + death_team['avg_score']

In [264]:
death_team['true_bpd'] = -sum(death_team['balls'])/sum(death_team['dismissals']) + death_team['bpd']
death_team['true_six_frequency'] = sum(death_team['balls'])/sum(death_team['sixes']) -death_team['six_frequency']
death_team['true_four_frequency'] = sum(death_team['balls'])/sum(death_team['fours']) - death_team['four_frequency']

In [265]:
death_team['avg_score_rank'] = death_team['avg_score'].rank(ascending=False).astype('int')
death_team['bpd_rank'] = death_team['bpd'].rank(ascending=False).astype('int')
death_team['six_freq_rank'] = death_team['six_frequency'].rank(ascending=True).astype('int')
death_team['four_freq_rank'] = death_team['four_frequency'].rank(ascending=True).astype('int')

In [266]:
df_2024['current_score'] = df_2024.groupby(['match_id','innings']).agg(current_runs_off_bat = ('runs_off_bat','cumsum')) + df_2024.groupby(['match_id','innings']).agg(current_runs_off_bat = ('extras','cumsum'))

In [267]:
df_2024['player_dismissed_flag'] = df_2024['player_dismissed'].apply(lambda x:0 if pd.isna(x) else 1)

In [268]:
df_2024['current_wickets'] = df_2024.groupby(['match_id','innings']).agg({'player_dismissed_flag':'cumsum'})

In [269]:
death_team[['death_entry_runs_1stinns','death_entry_wickets_1stinns']] = df_2024[(df_2024["ball"]==15.1) & (df_2024["innings"]==1)].groupby(['batting_team']).agg({'current_score':'mean',
                                                                            'current_wickets':'mean'}).round().astype('int')
                                                                                       

In [270]:
death_team

,runs,balls,dismissals,avg_score,bpd,sixes,fours,six_frequency,four_frequency,true_avg_score,true_bpd,true_six_frequency,true_four_frequency,avg_score_rank,bpd_rank,six_freq_rank,four_freq_rank,death_entry_runs_1stinns,death_entry_wickets_1stinns
batting_team,,,,,,,,,,,,,,,,,,,
Chennai Super Kings,742,388,24,57.371134,16.166667,48,50,8.083333,7.760000,1.320459,3.941077,0.838043,-0.179624,3,1,4,6,131,3
Delhi Capitals,641,353,29,54.475921,12.172414,39,45,9.051282,7.844444,-1.574754,-0.053175,-0.129906,-0.264069,6,6,7,7,142,4
Gujarat Titans,581,325,30,53.630769,10.833333,24,55,13.541667,5.909091,-2.419906,-1.392256,-4.620291,1.671285,8,7,10,1,129,4
Kolkata Knight Riders,562,269,27,62.676580,9.962963,38,39,7.078947,6.897436,6.625905,-2.262626,1.842429,0.682940,2,9,1,4,153,4
Lucknow Super Giants,668,384,24,52.187500,16.000000,29,56,13.241379,6.857143,-3.863175,3.774411,-4.320003,0.723233,10,2,9,3,129,4
Mumbai Indians,648,346,36,56.184971,9.611111,44,37,7.863636,9.351351,0.134296,-2.614478,1.057740,-1.770976,4,10,3,9,122,4
Punjab Kings,673,364,28,55.467033,13.000000,41,43,8.878049,8.465116,-0.583642,0.774411,0.043327,-0.884740,5,4,6,8,120,4
Rajasthan Royals,719,397,28,54.332494,14.178571,42,53,9.452381,7.490566,-1.718181,1.952982,-0.531005,0.089810,7,3,8,5,118,4
Royal Challengers Bengaluru,816,390,38,62.769231,10.263158,55,61,7.090909,6.393443,6.718556,-1.962431,1.830467,1.186933,1,8,2,2,135,4


In [271]:
df_2024_death['over_no'] = df_2024_death['ball'].apply(lambda x: str(x).split('.')[0])
df_2024_death['ball_no'] = df_2024_death['ball'].apply(lambda x: str(x).split('.')[1])

In [272]:
death_by_over = pd.DataFrame(df_2024_death.groupby(['batting_team','match_id','over_no'])['runs_off_bat'].sum()).reset_index()

In [273]:
death_by_over['15+_flag'] = death_by_over['runs_off_bat'].apply(lambda x:1 if x>=15 else 0)

In [274]:
death_above_15 = death_by_over.groupby(['batting_team']).agg(runs_over_15=('15+_flag','sum'),total_occ=('runs_off_bat','count'))

In [275]:
death_team['runs_over_15_percent'] = death_above_15['runs_over_15']/death_above_15['total_occ']*100

In [276]:
death_team['runs_over_15_rank'] = death_team['runs_over_15_percent'].rank(ascending=False).astype('int')
death_team['death_entry_runs_1stinns_rank'] = death_team['death_entry_runs_1stinns'].rank(ascending=False).astype('int')

In [277]:
death_team

,runs,balls,dismissals,avg_score,bpd,sixes,fours,six_frequency,four_frequency,true_avg_score,true_bpd,true_six_frequency,true_four_frequency,avg_score_rank,bpd_rank,six_freq_rank,four_freq_rank,death_entry_runs_1stinns,death_entry_wickets_1stinns,runs_over_15_percent,runs_over_15_rank,death_entry_runs_1stinns_rank
batting_team,,,,,,,,,,,,,,,,,,,,,,
Chennai Super Kings,742,388,24,57.371134,16.166667,48,50,8.083333,7.760000,1.320459,3.941077,0.838043,-0.179624,3,1,4,6,131,3,21.212121,6,5
Delhi Capitals,641,353,29,54.475921,12.172414,39,45,9.051282,7.844444,-1.574754,-0.053175,-0.129906,-0.264069,6,6,7,7,142,4,24.193548,3,3
Gujarat Titans,581,325,30,53.630769,10.833333,24,55,13.541667,5.909091,-2.419906,-1.392256,-4.620291,1.671285,8,7,10,1,129,4,19.298246,7,6
Kolkata Knight Riders,562,269,27,62.676580,9.962963,38,39,7.078947,6.897436,6.625905,-2.262626,1.842429,0.682940,2,9,1,4,153,4,30.434783,1,1
Lucknow Super Giants,668,384,24,52.187500,16.000000,29,56,13.241379,6.857143,-3.863175,3.774411,-4.320003,0.723233,10,2,9,3,129,4,18.181818,8,6
Mumbai Indians,648,346,36,56.184971,9.611111,44,37,7.863636,9.351351,0.134296,-2.614478,1.057740,-1.770976,4,10,3,9,122,4,23.728814,4,8
Punjab Kings,673,364,28,55.467033,13.000000,41,43,8.878049,8.465116,-0.583642,0.774411,0.043327,-0.884740,5,4,6,8,120,4,22.222222,5,9
Rajasthan Royals,719,397,28,54.332494,14.178571,42,53,9.452381,7.490566,-1.718181,1.952982,-0.531005,0.089810,7,3,8,5,118,4,16.176471,10,10
Royal Challengers Bengaluru,816,390,38,62.769231,10.263158,55,61,7.090909,6.393443,6.718556,-1.962431,1.830467,1.186933,1,8,2,2,135,4,27.272727,2,4


In [278]:
death_team[['avg_score','bpd','six_frequency','four_frequency','death_entry_runs_1stinns','death_entry_wickets_1stinns',
           'runs_over_15_percent']]

,avg_score,bpd,six_frequency,four_frequency,death_entry_runs_1stinns,death_entry_wickets_1stinns,runs_over_15_percent
batting_team,,,,,,,
Chennai Super Kings,57.371134,16.166667,8.083333,7.760000,131,3,21.212121
Delhi Capitals,54.475921,12.172414,9.051282,7.844444,142,4,24.193548
Gujarat Titans,53.630769,10.833333,13.541667,5.909091,129,4,19.298246
Kolkata Knight Riders,62.676580,9.962963,7.078947,6.897436,153,4,30.434783
Lucknow Super Giants,52.187500,16.000000,13.241379,6.857143,129,4,18.181818
Mumbai Indians,56.184971,9.611111,7.863636,9.351351,122,4,23.728814
Punjab Kings,55.467033,13.000000,8.878049,8.465116,120,4,22.222222
Rajasthan Royals,54.332494,14.178571,9.452381,7.490566,118,4,16.176471
Royal Challengers Bengaluru,62.769231,10.263158,7.090909,6.393443,135,4,27.272727


In [279]:
death_team[['true_avg_score','true_bpd','true_six_frequency','true_four_frequency',
           'death_entry_runs_1stinns','death_entry_wickets_1stinns',
           'runs_over_15_percent']]

,true_avg_score,true_bpd,true_six_frequency,true_four_frequency,death_entry_runs_1stinns,death_entry_wickets_1stinns,runs_over_15_percent
batting_team,,,,,,,
Chennai Super Kings,1.320459,3.941077,0.838043,-0.179624,131,3,21.212121
Delhi Capitals,-1.574754,-0.053175,-0.129906,-0.264069,142,4,24.193548
Gujarat Titans,-2.419906,-1.392256,-4.620291,1.671285,129,4,19.298246
Kolkata Knight Riders,6.625905,-2.262626,1.842429,0.682940,153,4,30.434783
Lucknow Super Giants,-3.863175,3.774411,-4.320003,0.723233,129,4,18.181818
Mumbai Indians,0.134296,-2.614478,1.057740,-1.770976,122,4,23.728814
Punjab Kings,-0.583642,0.774411,0.043327,-0.884740,120,4,22.222222
Rajasthan Royals,-1.718181,1.952982,-0.531005,0.089810,118,4,16.176471
Royal Challengers Bengaluru,6.718556,-1.962431,1.830467,1.186933,135,4,27.272727


In [280]:
death_team[['avg_score_rank','bpd_rank','six_freq_rank','four_freq_rank','death_entry_runs_1stinns_rank',
           'runs_over_15_rank']]

,avg_score_rank,bpd_rank,six_freq_rank,four_freq_rank,death_entry_runs_1stinns_rank,runs_over_15_rank
batting_team,,,,,,
Chennai Super Kings,3,1,4,6,5,6
Delhi Capitals,6,6,7,7,3,3
Gujarat Titans,8,7,10,1,6,7
Kolkata Knight Riders,2,9,1,4,1,1
Lucknow Super Giants,10,2,9,3,6,8
Mumbai Indians,4,10,3,9,8,4
Punjab Kings,5,4,6,8,9,5
Rajasthan Royals,7,3,8,5,10,10
Royal Challengers Bengaluru,1,8,2,2,4,2


# Death batting: Player

In [281]:
death_batting_player = df_2024_death.groupby(["striker"]).agg(team=('batting_team','first'),runs=('runs_off_bat','sum'),
                                    balls=('ball','count'),wides=('wides','count'))

In [282]:
death_batting_player['balls'] = death_batting_player['balls'] - death_batting_player['wides']

In [283]:
death_batting_player.drop(columns={'wides'},inplace=True)

In [284]:
death_batting_player = death_batting_player.merge(df_2024_death[df_2024_death["runs_off_bat"]==4].groupby(["striker"]).agg(fours=('runs_off_bat','count')),on='striker',how='left').fillna(0)
death_batting_player = death_batting_player.merge(df_2024_death[df_2024_death["runs_off_bat"]==6].groupby(["striker"]).agg(sixes=('runs_off_bat','count')),on='striker',how='left').fillna(0)

In [285]:
death_batting_player['fours'] = death_batting_player['fours'].astype('int')
death_batting_player['sixes'] = death_batting_player['sixes'].astype('int')

In [286]:
death_batting_player['strike_rate'] = death_batting_player['runs']/death_batting_player['balls']*100

In [287]:
death_batting_player['four_frequency'] = death_batting_player['balls']/death_batting_player['fours']
death_batting_player['six_frequency'] = death_batting_player['balls']/death_batting_player['sixes']

In [288]:
death_batting_player['true_strike_rate'] = death_batting_player['strike_rate'] - sum(death_batting_player['runs'])/sum(death_batting_player['balls'])*100
death_batting_player['true_four_freq'] = -death_batting_player['four_frequency'] + sum(death_batting_player['balls'])/sum(death_batting_player['fours'])
death_batting_player['true_six_freq'] = -death_batting_player['six_frequency'] + sum(death_batting_player['balls'])/sum(death_batting_player['sixes'])

In [289]:
death_batting_player.sort_values(by='true_strike_rate',ascending=False).reset_index()

,striker,team,runs,balls,fours,sixes,strike_rate,four_frequency,six_frequency,true_strike_rate,true_four_freq,true_six_freq
0,WG Jacks,Royal Challengers Bengaluru,28,5,1,4,560.000000,5.0,1.25,386.243849,2.636743,7.737715
1,GJ Maxwell,Royal Challengers Bengaluru,16,5,2,1,320.000000,2.5,5.00,146.243849,5.136743,3.987715
2,L Wood,Mumbai Indians,9,3,0,1,300.000000,inf,3.00,126.243849,-inf,5.987715
3,DJ Hooda,Lucknow Super Giants,17,6,2,1,283.333333,3.0,6.00,109.577182,4.636743,2.987715
4,R Shepherd,Mumbai Indians,57,21,5,5,271.428571,4.2,4.20,97.672421,3.436743,4.787715
...,...,...,...,...,...,...,...,...,...,...,...,...
139,A Raghuvanshi,Kolkata Knight Riders,0,1,0,0,0.000000,inf,inf,-173.756151,-inf,-inf
140,Mayank Dagar,Royal Challengers Bengaluru,0,1,0,0,0.000000,inf,inf,-173.756151,-inf,-inf
141,MM Sharma,Gujarat Titans,0,1,0,0,0.000000,inf,inf,-173.756151,-inf,-inf
142,Washington Sundar,Sunrisers Hyderabad,0,1,0,0,0.000000,inf,inf,-173.756151,-inf,-inf


In [290]:
def strike_rate_5_ball(batter):
    df_2024_no_wide = df_2024_death[df_2024_death['wides'].isnull()]
    df_2024_no_wide[['cum_score','cum_balls']]=df_2024_no_wide.groupby(['match_id','striker']).agg({'runs_off_bat':'cumsum',
                                                                                                     'ball':'cumcount'})
    df_2024_no_wide['cum_balls']+=1
    df_5_ball = df_2024_no_wide[df_2024_no_wide['cum_balls']<=5]
    df_batter = df_5_ball[df_5_ball['striker']==batter]
    if df_batter.empty:
        return 0
    sr = sum(df_batter['runs_off_bat'])/len(df_batter)*100
    return round(sr,2)

In [291]:
all_batters = df_2024['striker'].unique().tolist()

In [292]:
all_batters_5_ball_sr = []
for batter in all_batters:
    all_batters_5_ball_sr.append(strike_rate_5_ball(batter))
    

In [293]:
def strike_rate_10_ball(batter):
    df_2024_no_wide = df_2024_death[df_2024_death['wides'].isnull()]
    df_2024_no_wide[['cum_score','cum_balls']]=df_2024_no_wide.groupby(['match_id','striker']).agg({'runs_off_bat':'cumsum',
                                                                                                     'ball':'cumcount'})
    df_2024_no_wide['cum_balls']+=1
    df_10_ball = df_2024_no_wide[df_2024_no_wide['cum_balls']<=10]
    df_batter = df_10_ball[df_10_ball['striker']==batter]
    if df_batter.empty:
        return 0
    sr = sum(df_batter['runs_off_bat'])/len(df_batter)*100
    return round(sr,2)

In [294]:
all_batters_10_ball_sr = []
for batter in all_batters:
    all_batters_10_ball_sr.append(strike_rate_10_ball(batter))
    

In [295]:
special_strike_rates = pd.DataFrame({'striker':all_batters,'5_ball_strike_rate':all_batters_5_ball_sr,
              '10_ball_strike_rate':all_batters_10_ball_sr})

In [296]:
death_batting_player = death_batting_player.merge(special_strike_rates,on='striker',how='left')

In [297]:
def consecutive_boundaries(df):
    if df.empty:
        return 0
    df = df[df['wides'].isnull()]
    df['boundary_flag'] = df['runs_off_bat'].apply(lambda x: 1 if (x==4)|(x==6) else 0)
    df['prev_boundary_flag'] = df['boundary_flag'].shift(1)
    df['prev_match_id'] = df['match_id'].shift(1)
    df['prev_striker'] = df['striker'].shift(1)
    df['is_consecutive_boundary'] = np.where((df['match_id']==df['prev_match_id']) & 
                                                        (df['striker']==df['prev_striker']) ,
                                                      ((df['boundary_flag']==1)&(df['prev_boundary_flag']==1)),np.nan) 
    df['consecutive_flag'] = df['is_consecutive_boundary'].apply(lambda x: 1 if x==True else 0)
    prob = sum(df['consecutive_flag']) / len(df['is_consecutive_boundary']) *100  
    return round(prob,2)

In [298]:
prob = []
for batter in all_batters:
    prob.append(consecutive_boundaries(df_2024_death[df_2024_death['striker']==batter]))
    

In [299]:
tment_avg = consecutive_boundaries(df_2024_death)
tment_avg

6.12

In [300]:
cons_boundaries = pd.DataFrame({'striker':all_batters,'consecutive_boundaries_probability':prob})

In [301]:
death_batting_player = death_batting_player.merge(cons_boundaries,on='striker',how='left')

In [302]:
death_batting_player['true_cbp'] = death_batting_player['consecutive_boundaries_probability'] - tment_avg 

In [303]:
death_batting_player_filter = death_batting_player[death_batting_player["balls"]>=25]

In [304]:
death_batting_player_filter.set_index('striker',inplace=True)

In [305]:
death_batting_player_filter[['team','runs','balls','strike_rate','true_strike_rate']].sort_values(by='true_strike_rate',ascending=False).reset_index().head(25)

,striker,team,runs,balls,strike_rate,true_strike_rate
0,T Stubbs,Delhi Capitals,252,96,262.500000,88.743849
1,Ramandeep Singh,Kolkata Knight Riders,68,28,242.857143,69.100992
2,Naman Dhir,Mumbai Indians,57,25,228.000000,54.243849
3,RR Pant,Delhi Capitals,133,59,225.423729,51.667578
4,Shashank Singh,Punjab Kings,135,61,221.311475,47.555325
5,MS Dhoni,Chennai Super Kings,161,73,220.547945,46.791794
6,KD Karthik,Royal Challengers Bengaluru,249,116,214.655172,40.899022
7,H Klaasen,Sunrisers Hyderabad,195,91,214.285714,40.529563
8,AD Russell,Kolkata Knight Riders,135,64,210.937500,37.181349
9,JC Buttler,Rajasthan Royals,72,35,205.714286,31.958135


In [306]:
death_batting_player_filter[['team','runs','balls','fours','four_frequency','true_four_freq']].sort_values(by='true_four_freq',ascending=False).reset_index().head(25)

,striker,team,runs,balls,fours,four_frequency,true_four_freq
0,R Tewatia,Gujarat Titans,158,90,21,4.285714,3.351029
1,MP Stoinis,Lucknow Super Giants,49,27,6,4.500000,3.136743
2,Rashid Khan,Gujarat Titans,70,44,9,4.888889,2.747854
3,Anuj Rawat,Royal Challengers Bengaluru,54,30,6,5.000000,2.636743
4,V Kohli,Royal Challengers Bengaluru,97,50,10,5.000000,2.636743
5,MS Dhoni,Chennai Super Kings,161,73,14,5.214286,2.422458
6,T Stubbs,Delhi Capitals,252,96,18,5.333333,2.303410
7,DA Miller,Gujarat Titans,110,54,10,5.400000,2.236743
8,R Powell,Rajasthan Royals,95,52,9,5.777778,1.858965
9,C Green,Royal Challengers Bengaluru,117,58,10,5.800000,1.836743


In [307]:
death_batting_player_filter[['team','runs','balls','sixes','six_frequency','true_six_freq']].sort_values(by='true_six_freq',ascending=False).reset_index().head(25)

,striker,team,runs,balls,sixes,six_frequency,true_six_freq
0,Ramandeep Singh,Kolkata Knight Riders,68,28,7,4.000000,4.987715
1,T Stubbs,Delhi Capitals,252,96,20,4.800000,4.187715
2,Naman Dhir,Mumbai Indians,57,25,5,5.000000,3.987715
3,RR Pant,Delhi Capitals,133,59,11,5.363636,3.624079
4,Shashank Singh,Punjab Kings,135,61,11,5.545455,3.442260
5,S Dube,Chennai Super Kings,177,89,16,5.562500,3.425215
6,MS Dhoni,Chennai Super Kings,161,73,13,5.615385,3.372330
7,H Klaasen,Sunrisers Hyderabad,195,91,16,5.687500,3.300215
8,KD Karthik,Royal Challengers Bengaluru,249,116,20,5.800000,3.187715
9,AD Russell,Kolkata Knight Riders,135,64,11,5.818182,3.169533


In [308]:
death_batting_player_filter[['team','runs','balls','strike_rate','true_strike_rate','5_ball_strike_rate','10_ball_strike_rate']].sort_values(by='10_ball_strike_rate',ascending=False).reset_index().head(25)

,striker,team,runs,balls,strike_rate,true_strike_rate,5_ball_strike_rate,10_ball_strike_rate
0,Ramandeep Singh,Kolkata Knight Riders,68,28,242.857143,69.100992,243.48,242.86
1,T Stubbs,Delhi Capitals,252,96,262.500000,88.743849,200.00,242.65
2,C Green,Royal Challengers Bengaluru,117,58,201.724138,27.967987,264.00,222.22
3,JC Buttler,Rajasthan Royals,72,35,205.714286,31.958135,260.00,210.00
4,KD Karthik,Royal Challengers Bengaluru,249,116,214.655172,40.899022,203.57,209.68
5,MS Dhoni,Chennai Super Kings,161,73,220.547945,46.791794,218.42,208.06
6,Shashank Singh,Punjab Kings,135,61,221.311475,47.555325,200.00,207.55
7,SS Iyer,Kolkata Knight Riders,113,56,201.785714,28.029563,188.24,205.88
8,Abdul Samad,Sunrisers Hyderabad,121,60,201.666667,27.910516,200.00,205.17
9,DA Miller,Gujarat Titans,110,54,203.703704,29.947553,192.00,202.22


In [309]:
death_batting_player_filter[['team','runs','balls','fours','sixes','consecutive_boundaries_probability','true_cbp']].sort_values(by='consecutive_boundaries_probability',ascending=False).reset_index().head(25)

,striker,team,runs,balls,fours,sixes,consecutive_boundaries_probability,true_cbp
0,T Stubbs,Delhi Capitals,252,96,18,20,18.75,12.63
1,Ramandeep Singh,Kolkata Knight Riders,68,28,3,7,14.29,8.17
2,Anuj Rawat,Royal Challengers Bengaluru,54,30,6,3,13.33,7.21
3,RR Pant,Delhi Capitals,133,59,9,11,11.86,5.74
4,Abdul Samad,Sunrisers Hyderabad,121,60,10,8,11.67,5.55
5,MP Stoinis,Lucknow Super Giants,49,27,6,2,11.11,4.99
6,LS Livingstone,Punjab Kings,71,38,6,5,10.53,4.41
7,R Parag,Rajasthan Royals,141,76,11,10,10.53,4.41
8,Shashank Singh,Punjab Kings,135,61,9,11,9.84,3.72
9,MS Dhoni,Chennai Super Kings,161,73,14,13,9.59,3.47


In [310]:
death_batting_player_filter[death_batting_player_filter['team']=='Kolkata Knight Riders']

,team,runs,balls,fours,sixes,strike_rate,four_frequency,six_frequency,true_strike_rate,true_four_freq,true_six_freq,5_ball_strike_rate,10_ball_strike_rate,consecutive_boundaries_probability,true_cbp
striker,,,,,,,,,,,,,,,
AD Russell,Kolkata Knight Riders,135,64,11,11,210.937500,5.818182,5.818182,37.181349,1.818561,3.169533,203.70,193.62,9.38,3.26
RK Singh,Kolkata Knight Riders,86,48,7,5,179.166667,6.857143,9.600000,5.410516,0.779600,-0.612285,168.57,179.17,6.25,0.13
Ramandeep Singh,Kolkata Knight Riders,68,28,3,7,242.857143,9.333333,4.000000,69.100992,-1.696590,4.987715,243.48,242.86,14.29,8.17
SS Iyer,Kolkata Knight Riders,113,56,8,8,201.785714,7.000000,7.000000,28.029563,0.636743,1.987715,188.24,205.88,7.14,1.02
VR Iyer,Kolkata Knight Riders,60,43,4,4,139.534884,10.750000,10.750000,-34.221267,-3.113257,-1.762285,129.17,147.22,2.33,-3.79


In [311]:
#case study: Ramandeep/Russell -- reference from V Iyer's 
#case study(pp): Narine/Salt/V Iyer 
#home vs away analysis
#bowling analysis (phasewise: team/individual/case with use of advanced performance metrics)
#Cleaning the code file
#dashboarding
#post : Linkedin/Medium/others*
#apply


# Ramandeep Singh

In [312]:
raman = df_2024[(df_2024["striker"]=='Ramandeep Singh')| (df_2024['non_striker']=='Ramandeep Singh')]

In [313]:
raman[raman['innings']==2]

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,current_score,player_dismissed_flag,current_wickets,over_no,ball_no


In [314]:
raman_by_match = raman.groupby(['match_id']).agg(innings=('innings','first'),entry_point=('ball','min'),
                                                 exit_point=('ball','max'))

In [315]:
raman_strike = raman[raman['striker']=='Ramandeep Singh'].groupby(['match_id']).agg(runs=('runs_off_bat','sum'),balls=('ball','count'), 
wides=('wides','count'))
raman_strike['balls'] = raman_strike['balls'] - raman_strike['wides']
raman_strike.drop(columns={'wides'},inplace=True)
raman_by_match[['runs','balls']] = raman_strike

In [316]:
raman_non_striker = raman[raman['non_striker']=='Ramandeep Singh'].groupby(['match_id']).agg(runs=('runs_off_bat','sum'),
balls=('ball','count'),wides=('wides','count'))
raman_non_striker['balls'] = raman_non_striker['balls'] - raman_non_striker['wides']
raman_non_striker.drop(columns={'wides'},inplace=True)
raman_by_match = raman_by_match.merge(raman_non_striker,on='match_id',how='left',suffixes=('_raman','_non_striker')).fillna(0)

In [317]:
raman_by_match['strike_rate_raman'] = raman_by_match['runs_raman']/raman_by_match['balls_raman']*100
raman_by_match['strike_rate_non_striker'] = raman_by_match['runs_non_striker']/raman_by_match['balls_non_striker']*100

In [318]:
raman_by_match[['runs_non_striker','balls_non_striker']] = raman_by_match[['runs_non_striker','balls_non_striker']].astype('int')

In [319]:
raman_by_match.fillna(0,inplace=True)
raman_by_match
#line graph

,innings,entry_point,exit_point,runs_raman,balls_raman,runs_non_striker,balls_non_striker,strike_rate_raman,strike_rate_non_striker
match_id,,,,,,,,,
1422121,1,7.4,12.1,35,17,18,12,205.882353,150.000000
1422134,1,19.2,19.3,2,2,0,0,100.000000,0.000000
1426260,1,8.3,11.5,13,12,8,9,108.333333,88.888889
1426269,1,19.3,19.6,1,1,7,3,100.000000,233.333333
1426274,1,17.4,19.7,24,9,16,7,266.666667,228.571429
1426280,1,19.3,19.7,6,3,0,1,200.000000,0.000000
1426289,1,17.1,17.4,2,4,0,0,50.000000,0.000000
1426292,1,18.1,19.7,25,6,9,6,416.666667,150.000000
1426298,1,13.1,15.6,17,8,14,10,212.500000,140.000000


# Andre Russell

In [320]:
russell = df_2024[(df_2024["striker"]=='AD Russell')|(df_2024['non_striker']=='AD Russell')]

In [321]:
russell_by_match = russell.groupby(['match_id']).agg(innings=('innings','first'),entry_point=('ball','min'),
                                                 exit_point=('ball','max'))

In [322]:
russell_by_match

,innings,entry_point,exit_point
match_id,,,
1422121,1,13.6,19.6
1422134,1,12.5,19.1
1426260,1,16.5,18.3
1426269,1,13.1,16.1
1426274,1,13.2,19.7
1426280,1,12.4,15.6
1426289,1,16.4,16.7
1426292,1,12.1,14.3
1426298,1,8.2,12.6


In [323]:
russell_strike = russell[russell['striker']=='AD Russell'].groupby(['match_id']).agg(runs=('runs_off_bat','sum'),balls=('ball','count'), 
wides=('wides','count'))
russell_strike['balls'] = russell_strike['balls'] - russell_strike['wides']
russell_strike.drop(columns={'wides'},inplace=True)
russell_by_match[['runs','balls']] = russell_strike

In [324]:
russell_by_match

,innings,entry_point,exit_point,runs,balls
match_id,,,,,
1422121,1,13.6,19.6,64,25
1422134,1,12.5,19.1,41,19
1426260,1,16.5,18.3,10,10
1426269,1,13.1,16.1,13,10
1426274,1,13.2,19.7,27,20
1426280,1,12.4,15.6,24,12
1426289,1,16.4,16.7,7,2
1426292,1,12.1,14.3,12,8
1426298,1,8.2,12.6,24,14


In [325]:
russell_non_striker = russell[russell['non_striker']=='AD Russell'].groupby(['match_id']).agg(runs=('runs_off_bat','sum'),
balls=('ball','count'),wides=('wides','count'))
russell_non_striker['balls'] = russell_non_striker['balls'] - russell_non_striker['wides']
russell_non_striker.drop(columns={'wides'},inplace=True)
russell_by_match = russell_by_match.merge(russell_non_striker,on='match_id',how='left',suffixes=('_russell','_non_striker')).fillna(0)

In [326]:
russell_by_match['strike_rate_russell'] = russell_by_match['runs_russell']/russell_by_match['balls_russell']*100
russell_by_match['strike_rate_non_striker'] = russell_by_match['runs_non_striker']/russell_by_match['balls_non_striker']*100
russell_by_match
#does not make a lot of sense, a lot of times russell was slower 

,innings,entry_point,exit_point,runs_russell,balls_russell,runs_non_striker,balls_non_striker,strike_rate_russell,strike_rate_non_striker
match_id,,,,,,,,,
1422121,1,13.6,19.6,64,25,21,13,256.000000,161.538462
1422134,1,12.5,19.1,41,19,49,22,215.789474,222.727273
1426260,1,16.5,18.3,10,10,2,1,100.000000,200.000000
1426269,1,13.1,16.1,13,10,29,9,130.000000,322.222222
1426274,1,13.2,19.7,27,20,45,23,135.000000,195.652174
1426280,1,12.4,15.6,24,12,13,6,200.000000,216.666667
1426289,1,16.4,16.7,7,2,6,2,350.000000,300.000000
1426292,1,12.1,14.3,12,8,7,6,150.000000,116.666667
1426298,1,8.2,12.6,24,14,23,15,171.428571,153.333333


In [327]:
#analysis of what happened when 1. raman is batting ,2. russell is batting, 3. both bat together, 4. neither bat together

In [328]:
sum(raman_by_match['runs_raman'])/sum(raman_by_match['balls_raman'])*100

201.61290322580646

In [329]:
sum(raman['runs_off_bat'])/(len(raman) - raman['wides'].sum())*100

179.0909090909091

In [330]:
sum(russell_by_match['runs_russell'])/sum(russell_by_match['balls_russell'])*100

185.0

In [331]:
sum(russell['runs_off_bat'])/(len(russell) - russell['wides'].sum())*100

200.48076923076925

In [332]:
raman_and_russell = (df_2024[(df_2024['striker'].isin(['Ramandeep Singh','AD Russell']))&(df_2024['non_striker'].isin(['Ramandeep Singh','AD Russell']))])

In [333]:
raman_and_russell['runs_off_bat'].sum()/(len(raman_and_russell)-raman_and_russell['wides'].sum())*100

250.0

In [334]:
not_raman_russell = df_2024[(~df_2024['striker'].isin(['Ramandeep Singh','AD Russell'])) & (df_2024['batting_team']=='Kolkata Knight Riders') 
       & (~df_2024['non_striker'].isin(['Ramandeep Singh','AD Russell'])) & (df_2024['ball']>=13)]

In [335]:
not_raman_russell['runs_off_bat'].sum()/(len(not_raman_russell)-not_raman_russell['wides'].sum())*100

155.3921568627451

In [336]:
#iska dataframe banao - 6 values, line graph

# Conversions to CSV

In [337]:
team_pp_2024_bat.to_csv('team_pp_2024_bat.csv')

In [338]:
player_pp_2024_bat_final.to_csv('player_pp_2024_bat_final.csv')